# **Big Data Modeling and Management Assigment - Homework 1**

# **Submission**

GROUP NUMBER: **5**

GROUP MEMBERS:

|STUDENT NAME|STUDENT NUMBER|
|:---:|:---:|
|Alexandre Gonçalves|20240738|
|André Silvestre|20240502|
|Filipa Pereira|20240509|
|João Henriques|20240499|
|Umeima Mahomed|20240543|

## **🍺 The Beer project  🍺** 

As it was shown in classes, graph databases are a natural way of navegating related information. For this first project we will be taking a graph database to analyse beer and breweries!   

The project datasets are based on [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. 

### **Problem description**

Imagine you are working in the Data Management department of Analytics company.
Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions while adjusting the database to meet the needs of your colleagues.
Please record and keep track of your database changes, and submit the file with all cells run and with the output shown.

### **Questions**

1. Explore the database: get familiar with current schema, elements and other important database parameters. [1 point]
2. Adjust the database and mention reasoning behind: e.g. clean errors, remove redundancies, adjust schema as necessary. Visualize the final version of database schema. [4 points]
3. Analytics department requires the following information for the biweekly reporting: [5 points]
    1. How many reviews has the beer with the most reviews?
    2. Which three users wrote the most reviews about beers?
    3. Find all beers that are described with following words: 'fruit', 'complex', 'nutty', 'dark'.
    4. Which top three breweries produce the largest variety of beer styles?
    5. Which country produces the most beer styles?
4. Market Analysis department in your company accesses and updates the trends data on the daily basis. Given that, consider how you need to optimize the database and its performance so that the following queries are efficient. Measure performance to communicate your improvements using PROFILE before final query. Answer the following: [4 points]
    1. Using ABV score, find five strongest beers, display their ABV score and the corresponding brewery? Keep in mind that the strongest known beer is Snake Venom, and deal with the error entries in the database.
    2. Using the answer from question 2, find the top 5 distict beer styles with the highest average score of smell + feel that were reviewed by the third most productive user. Keep in mind that cleaning the database earlier should ensure correct results.
5. Answer **two out of four** of the following questions using Graph Algorithms (gds): [NB: make sure to clear the graph before using it again] For the quarterly report, Analytics department the follownig information. [6 points]
    1. Which two countries are most similiar when it comes to their top five most produced Beer styles?
    2. Which beer is the most popular when considering the number of users who reviewed it? 
    3. Users are connected together by their reviews of beers, taking into consideration the "smell" score they assign as a weight, how many communities are formed from these relationships? How many users are in the three largest communities? 
    4. Which user is the most influential when it comes to reviews of distinct beers by style?
 
### **Groups** 

Groups should have 4 people maximum. Please mark which group you are here: https://shorturl.at/zE0QP 

### **Submission**      

The code used to produce the results and to-the-point explations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself. Preferably one Jupyter notebook per group.

Delivery date: Until the **midnight of March 18, 2025**

### **Evaluation**   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and efficiency of the query (based on database schema).  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

## **Loading the Database**

Be sure that you **don't have** the neo4j docker container from the classes running (you can Stop it in the desktop app or with the command "`docker stop Neo4JLab`")


The default container does not have any data whatsoever, we will have to load a database into our docker image:
- Download and unzip the `Neo4JHWData` file provided in Moodle.
- Copy the path of the `Neo4JHWData` folder of the unziped file, e.g. `C:/PATH/Neo4JHWData/data`.
- Download and unzip the `Neo4JPlugins` file provided in Moodle.
- Copy the path of the `Neo4JPlugins` folder of the unziped file, e.g. `C:/PATH/Neo4Jplugins`.
- Change the code below accordingly. As you might have noticed, you do not have a user called `nunoa`, please use the appropriate path that you got from the previous step. Be sure that you have a neo4j docker container running: \

`docker run --name Neo4JHW2025 -p 7474:7474 -p 7687:7687 -d -v "c:\PATH\Neo4JPlugins":/plugins -v "c:\PATH\Neo4JHWData\data":/data --env NEO4J_AUTH=neo4j/test --env NEO4J_dbms_connector_https_advertised__address="localhost:7473" --env NEO4J_dbms_connector_http_advertised__address="localhost:7474" --env NEO4J_dbms_connector_bolt_advertised__address="localhost:7687" --env NEO4J_dbms_security_procedures_unrestricted=gds.* --env NEO4J_dbms_security_procedures_allowlist="gds.*" neo4j:4.4.5`

- Since Neo4j is trying to recognize a new database folder, this might take a bit (let's say 3 minutes), so don't worry.

If the neo4j browser fails to load gds plugins, run the following in the Command Prompt before creating the container again:
`// Remove stopped containers //
docker container prune -f
// Remove unused images //
docker image prune -a -f
// Remove unused volumes //
docker volume prune -f
// Remove unused networks //
docker network prune -f
// Remove all unused resources in one command //
docker system prune -a -f`

<div class="alert alert-block alert-info">

`docker run --name Neo4JHW2025 -p 7474:7474 -p 7687:7687 -d -v "C:\Users\André Silvestre\OneDrive - NOVAIMS\MDSAA-DS_1ºAno\2º Semestre\5E_Big Data Modelling and Managment_[BDMM]\[BDMM]_Assignment1&2\data\Neo4JPlugins":/plugins -v "C:\Users\André Silvestre\OneDrive - NOVAIMS\MDSAA-DS_1ºAno\2º Semestre\5E_Big Data Modelling and Managment_[BDMM]\[BDMM]_Assignment1&2\data\NEW_VERSION\data":/data --env NEO4J_AUTH=neo4j/test --env NEO4J_dbms_connector_https_advertised__address="localhost:7473" --env NEO4J_dbms_connector_http_advertised__address="localhost:7474" --env NEO4J_dbms_connector_bolt_advertised__address="localhost:7687" --env NEO4J_dbms_security_procedures_unrestricted=gds.* --env NEO4J_dbms_security_procedures_allowlist="gds.*" neo4j:4.4.5`

</div>

In [1]:
from neo4j import GraphDatabase
from pprint import pprint

# Extra
import pandas as pd               # For data manipulation
import numpy as np                # For numerical operations
from tqdm import tqdm             # Progress bar

# Disable FutureWarning messages
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
NEO4J_URI="neo4j://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="test"

In [3]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD), )

In [4]:
def execute_read(driver, query):    
    with driver.session(database="neo4j") as session:
        result = session.execute_read(lambda tx, query: list(tx.run(query)), query)
    return result

In [5]:
def execute_write(driver, query):
    with driver.session(database="neo4j") as session:
        # Write transactions allow the driver to handle retries and transient errors
        result = session.execute_write(lambda tx, query: list(tx.run(query)), query)
    return result

---

# **👨‍💻 Answering the questions**

## **1. Explore the database**

In [6]:
# 1. Explore the database: get familiar with current schema, elements and other important database parameters. [1 point]

# 1.1. Get the labels of all nodes in the database
query = """
    CALL db.labels()
"""
result = execute_read(driver, query)
print("Labels of all nodes in the database:")
pprint(result)

Labels of all nodes in the database:
[<Record label='COUNTRIES'>,
 <Record label='CITIES'>,
 <Record label='BREWERIES'>,
 <Record label='BEERS'>,
 <Record label='REVIEWS'>,
 <Record label='STYLE'>,
 <Record label='USER'>]


In [7]:
# 1.2. Get the relationship types in the database
query = """
    CALL db.relationshipTypes()
"""
result = execute_read(driver, query)
print("Relationship types in the database:")
pprint(result)

Relationship types in the database:
[<Record relationshipType='REVIEWED'>,
 <Record relationshipType='BREWED'>,
 <Record relationshipType='IN'>,
 <Record relationshipType='HAS_STYLE'>,
 <Record relationshipType='POSTED'>]


In [8]:
# 1.3. Get the number of nodes in the database
query = """
    MATCH (n)
    RETURN count(n) as count
"""
result = execute_read(driver, query)
pprint(result)

[<Record count=3215489>]


In [9]:
# 1.4. Get the number of relationships in the database
query = """
    MATCH ()-[r]->()
    RETURN count(r) as count
"""
result = execute_read(driver, query)
pprint(result)

[<Record count=5856205>]


In [10]:
# 1.5. Get the number of nodes per label in the database
query = """
    MATCH (n)
    RETURN labels(n) as label, count(n) as count
"""
result = execute_read(driver, query)
pprint(result)

[<Record label=['COUNTRIES'] count=400>,
 <Record label=['CITIES'] count=23330>,
 <Record label=['BREWERIES'] count=100694>,
 <Record label=['BEERS'] count=417746>,
 <Record label=['REVIEWS'] count=2549252>,
 <Record label=['STYLE'] count=113>,
 <Record label=['USER'] count=123935>]


- With this query we can see that we have $7$ different types of nodes in the database: **`COUNTRIES`** with $400$ nodes, **`CITIES`** with $23\;330$ nodes, **`BREWERIES`** with $100\;694$ nodes, **`BEERS`** with $417\;746$ nodes, **`REVIEWS`** with $2\;549\;252$ nodes, **`STYLE`** with $113$ nodes and **`USER`** with $123\;935$ nodes.

In [11]:
# 1.6. Get the number of relationships per type in the database
query = """
    MATCH ()-[r]->()
    RETURN type(r) as type, count(r) as NumberOfRelationships
    ORDER BY NumberOfRelationships DESC
"""
result = execute_read(driver, query)
pprint(result)

[<Record type='POSTED' NumberOfRelationships=2538044>,
 <Record type='REVIEWED' NumberOfRelationships=2537991>,
 <Record type='BREWED' NumberOfRelationships=358873>,
 <Record type='HAS_STYLE' NumberOfRelationships=358873>,
 <Record type='IN' NumberOfRelationships=62424>]


- We also have $5$ different types of relationships in the database: **`POSTED`** with $2\;538\;044$ relationships, **`REVIEWED`** with $2\;537\;991$ relationships, **`BREWED`** with $358\;873$ relationships, **`HAS_STYLE`** with $358\;873$ relationships and **`IN`** with $62\;424$ relationships.

In [12]:
# 1.7. For each node label, get a list of properties and their types

# This query uses the built-in procedure db.schema.nodeTypeProperties() available in Neo4j 4.x+
# It returns, for each node label (nodeType), each property (propertyName) and its type(s) (propertyTypes)
# Source: https://neo4j.com/docs/operations-manual/current/procedures/#procedure_db_schema_nodetypeproperties
query = """
    CALL db.schema.nodeTypeProperties() YIELD nodeType, propertyName, propertyTypes, mandatory
    RETURN nodeType, propertyName, propertyTypes, mandatory
    ORDER BY nodeType, propertyName
"""
result = execute_read(driver, query)
print("Properties and types for each node label:")
for rec in result:
    print(f"Node Label: {rec['nodeType']:<15} | Property: {rec['propertyName']:<15} | Type(s): {', '.join(rec['propertyTypes']):<20} | Mandatory: {rec['mandatory']}")

Properties and types for each node label:
Node Label: :`BEERS`        | Property: abv             | Type(s): String               | Mandatory: True
Node Label: :`BEERS`        | Property: availability    | Type(s): String               | Mandatory: True
Node Label: :`BEERS`        | Property: brewery_id      | Type(s): String               | Mandatory: True
Node Label: :`BEERS`        | Property: id              | Type(s): String               | Mandatory: True
Node Label: :`BEERS`        | Property: name            | Type(s): String               | Mandatory: True
Node Label: :`BEERS`        | Property: notes           | Type(s): String               | Mandatory: True
Node Label: :`BEERS`        | Property: retired         | Type(s): String               | Mandatory: True
Node Label: :`BEERS`        | Property: state           | Type(s): String               | Mandatory: True
Node Label: :`BREWERIES`    | Property: id              | Type(s): String               | Mandatory: True
Node

In [13]:
# 1.7.1. Get a example of 1 node for each label
query = """
    MATCH (n)
    WITH DISTINCT labels(n) AS label, collect(n) AS nodes
    RETURN label, nodes[0] AS exampleNode
"""

# Explanation:
# 1. MATCH (n):                                             Match all nodes in the database
# 2. WITH DISTINCT labels(n) AS label, collect(n) AS nodes: For each node, get its labels and collect the node itself
# 3. RETURN label, nodes[0] AS exampleNode:                 Return the label and an example node for each label

result = execute_read(driver, query)
print("Example of 1 node for each label:")
for rec in result:
    print(f"Node Label: {rec['label']} | Node: {rec['exampleNode']}")

Example of 1 node for each label:
Node Label: ['COUNTRIES'] | Node: <Node element_id='0' labels=frozenset({'COUNTRIES'}) properties={'name': 'BE'}>
Node Label: ['CITIES'] | Node: <Node element_id='200' labels=frozenset({'CITIES'}) properties={'name': 'Erpe-Mere'}>
Node Label: ['BREWERIES'] | Node: <Node element_id='11865' labels=frozenset({'BREWERIES'}) properties={'types': 'Brewery', 'notes': 'No notes at this time.', 'name': 'Brouwerij Danny', 'id': '19730', 'state': 'nan'}>
Node Label: ['BEERS'] | Node: <Node element_id='62212' labels=frozenset({'BEERS'}) properties={'notes': 'No notes at this time.', 'abv': '7.3', 'name': 'Olde Cogitator', 'retired': 'f', 'state': 'CA', 'id': '202522', 'availability': ' Rotating', 'brewery_id': '2199'}>
Node Label: ['REVIEWS'] | Node: <Node element_id='421086' labels=frozenset({'REVIEWS'}) properties={'date': '2017-12-21', 'score': '4.5', 'taste': '4.5', 'feel': '4.5', 'overall': '4.5', 'beer_id': '125646', 'text': '\xa0\xa0', 'id': '1', 'smell': '

- All **properties of the nodes** are in **`strings`** format, so we will have to convert them to the correct format in order to make the queries more efficient.

**List of Properties to convert:**
- **`abv`** in **`BEERS`** nodes to **`float`**.
- **`score`**, **`taste`**, **`feel`**, **`overall`**, **`smell`** and **`look`** in **`REVIEWS`** nodes to **`float`**.
- **`id`** in **`BEERS`**, **`BREWERIES`** and **`REVIEWS`** nodes to **`int`**.
- **`date`** in **`REVIEWS`** nodes to **`date`**.


In [14]:
# 1.8. For each relationship type, get a list of properties and their types (relTypeProperties)
query = """
    CALL db.schema.relTypeProperties() YIELD relType, propertyName, propertyTypes
    RETURN relType, propertyName, propertyTypes
    ORDER BY relType, propertyName
"""
result = execute_read(driver, query)
print("Properties and types for each relationship type:")
for rec in result:
    print("Relationship Type:", rec["relType"], "| Property:", rec["propertyName"], "| Type(s):", rec["propertyTypes"])

Properties and types for each relationship type:
Relationship Type: :`BREWED` | Property: None | Type(s): None
Relationship Type: :`HAS_STYLE` | Property: None | Type(s): None
Relationship Type: :`IN` | Property: None | Type(s): None
Relationship Type: :`POSTED` | Property: None | Type(s): None
Relationship Type: :`REVIEWED` | Property: None | Type(s): None


- All **relationships** don't have any properties.

In [15]:
# 1.9. For each pair of nodes, print the connection and its direction
query = """
    MATCH (a)-[r]->(b)
    WITH DISTINCT head(labels(a)) AS StartLabel, type(r) AS Relationship, head(labels(b)) AS EndLabel
    RETURN StartLabel, Relationship, EndLabel
    ORDER BY StartLabel, Relationship, EndLabel
"""
result = execute_read(driver, query)
print("Schema of connections (node label - relationship -> node label):")
for rec in result:
    print(rec["StartLabel"], "-", rec["Relationship"], "->", rec["EndLabel"])

Schema of connections (node label - relationship -> node label):
BEERS - HAS_STYLE -> STYLE
BEERS - REVIEWED -> REVIEWS
BREWERIES - BREWED -> BEERS
BREWERIES - IN -> CITIES
CITIES - IN -> COUNTRIES
REVIEWS - POSTED -> USER


#### **🧮 Summary of the Database Schema**

<center>

|     |   **Node Labels**  | **Properties**                                                     |**Total number of nodes**|
|:---:|:------------------:|:-------------------------------------------------------------------|:-----------------------:|
|**1**| `COUNTRIES`        | name                                                               |400                      |
|**2**| `CITIES`           | name                                                               |23330                    |
|**3**| `BREWERIES`        | notes, types, id, name, state                                      |100694                   |
|**4**| `BEERS`            | notes, abv, name, state, id, retired, availability, brewery_id     |417746                   |
|**5**| `REVIEWS`          | text, smell, look, taste, feel, overall, beer_id, id, date, score  |2549271                  | 
|**6**| `STYLE`            | name                                                               |113                      |
|**7**| `USER`             | name                                                               |123935                   |

<br><br>

|     |  **Relationship Types**  | **Total number of relationships** |
|:---:|:------------------------:|:---------------------------------:|
|**1**| `POSTED`                 |2538044                            |
|**2**| `REVIEWED`               |2537991                            |
|**3**| `BREWED`                 |358873                             |
|**4**| `HAS_STYLE`              |358873                             |
|**5**| `IN`                     |62424                              |



</center>

In [16]:
# 1.10 Visualize the schema of the database
query = """
    // What is related, and how
    CALL db.schema.visualization()
"""
result = execute_read(driver, query)
print("Schema visualization:")
pprint(result)

Schema visualization:
[<Record nodes=[<Node element_id='-5' labels=frozenset({'REVIEWS'}) properties={'name': 'REVIEWS', 'indexes': ['id'], 'constraints': []}>, <Node element_id='-4' labels=frozenset({'BEERS'}) properties={'name': 'BEERS', 'indexes': ['id'], 'constraints': []}>, <Node element_id='-1' labels=frozenset({'COUNTRIES'}) properties={'name': 'COUNTRIES', 'indexes': ['name'], 'constraints': []}>, <Node element_id='-3' labels=frozenset({'BREWERIES'}) properties={'name': 'BREWERIES', 'indexes': ['id'], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'STYLE'}) properties={'name': 'STYLE', 'indexes': ['name'], 'constraints': []}>, <Node element_id='-2' labels=frozenset({'CITIES'}) properties={'name': 'CITIES', 'indexes': ['name'], 'constraints': []}>, <Node element_id='-7' labels=frozenset({'USER'}) properties={'name': 'USER', 'indexes': ['name'], 'constraints': []}>] relationships=[<Relationship element_id='-1' nodes=(<Node element_id='-4' labels=frozenset({'BEERS'})

> Running the previous query in **`http://localhost:7474/browser/`** we get the following output:

<center><img src="./img/TheBeerProject_Schema.png" width="700"></center>

**NOTE:** Although the relationship **`IN`** between **`CITIES`** and itself is shown in the image, it is not present in the database as we can see in the previous query (**1.9.**).


In [17]:
# 1.11 Get a description of the database
#      Source of the query: http://localhost:7474/browser/ > Favorites > Sample Scripts > Data Profiling > What kind of nodes exist
query = """
    // What kind of nodes exist
    // Sample some nodes, reporting on property and relationship counts per node.
    MATCH (n) WHERE rand() <= 0.1
    RETURN
        DISTINCT labels(n),
            count(*) AS SampleSize,
            avg(size(keys(n))) as Avg_PropertyCount,
            min(size(keys(n))) as Min_PropertyCount,
            max(size(keys(n))) as Max_PropertyCount,
            avg(size( (n)-[]-() ) ) as Avg_RelationshipCount,
            min(size( (n)-[]-() ) ) as Min_RelationshipCount,
            max(size( (n)-[]-() ) ) as Max_RelationshipCount
"""

result = execute_read(driver, query)

# Convert the result to Pandas DataFrame for better visualization
data = [dict(record) for record in result]
df = pd.DataFrame(data)
df.set_index("labels(n)", inplace=True)
df.index.name = "Node Label"
print("Description of the database:")
df

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: } {title: This feature is deprecated and will be removed in future versions.} {description: A pattern expression should only be used in order to test the existence of a pattern. It should therefore only be used in contexts that evaluate to a boolean, e.g. inside the function exists() or in a WHERE-clause. All other uses are deprecated and should be replaced by a pattern comprehension.} {position: line: 11, column: 23, offset: 405} for query: '\n    // What kind of nodes exist\n    // Sample some nodes, reporting on property and relationship counts per node.\n    MATCH (n) WHERE rand() <= 0.1\n    RETURN\n        DISTINCT labels(n),\n            count(*) AS SampleSize,\n            avg(size(keys(n))) as Avg_PropertyCount,\n            min(size(keys(n))) as Min_PropertyCount,\n            max(size(keys(n))) as Max_PropertyCount,\n            avg(size( 

Description of the database:


,SampleSize,Avg_PropertyCount,Min_PropertyCount,Max_PropertyCount,Avg_RelationshipCount,Min_RelationshipCount,Max_RelationshipCount
Node Label,,,,,,,
[COUNTRIES],47,1.0,1,1,1.787234,0,28
[CITIES],2348,1.0,1,1,2.685264,0,222
[BREWERIES],10040,5.0,5,5,4.235857,0,899
[BEERS],41581,8.0,8,8,8.098891,0,3713
[REVIEWS],254240,10.0,10,10,1.991276,0,2
[STYLE],12,1.0,1,1,2381.416667,210,7729
[USER],12556,1.0,1,1,20.621695,0,3368


- **Orphan Nodes**: Nodes with `Min_RelationshipCount` of 0 are considered orphan nodes, meaning they are not connected to any other nodes in the graph. These nodes may represent incomplete data or entities that have not yet been linked to the rest of the dataset.
    - In this database, there are orphan nodes of type `COUNTRIES`, `CITIES`, `BREWERIES`, `BEERS`, `REVIEWS`, and `USER`.

- **`Min_PropertyCount` and `Max_PropertyCount`**: The consistency in `Min_PropertyCount` and `Max_PropertyCount` across all labels suggests that each node type has a uniform set of properties.

In [18]:
# 1.12 Get a summary statistics of each property for each node label in the database
#     Source of the query: https://neo4j.com/blog/developer/data-profiling-holistic-view-neo4j/
#                          https://neo4j.com/docs/cypher-manual/current/functions/aggregating/

# Get all properties for each node label
query = """
    CALL db.schema.nodeTypeProperties() YIELD nodeType, propertyName
    WITH nodeType, propertyName
    ORDER BY nodeType, propertyName
    RETURN nodeType, propertyName
"""
properties_result = execute_read(driver, query)

# Initialize a list to store statistics
stats_list = []

# Iterate over each property and calculate statistics
for record in tqdm(properties_result):
    nodeType = record['nodeType'].replace('`', '').replace(':', '')  # Remove backticks and ':'
    propertyName = record['propertyName']
    
    # Construct query for each property
    if propertyName.lower() in ['name', 'notes', 'state', 'availability', 'retired', 'types', 'date', 'text']:
        # String properties
        stats_query = f"""
            MATCH (n:{nodeType})
            RETURN
                '{nodeType}' AS nodeType,
                '{propertyName}' AS propertyName,
                COUNT(n.{propertyName}) AS count,
                COUNT(DISTINCT n.{propertyName}) AS distinctCount,
                '-' AS mean,
                '-' AS min,
                '-' AS max,
                COUNT(CASE WHEN n.{propertyName} IS NULL OR n.{propertyName} = '' OR toLower(trim(n.{propertyName})) IN ['nan', 'null', 'none', ' '] THEN 1 ELSE NULL END) AS missingCount
        """
    else:
        # Numeric properties
        stats_query = f"""
            MATCH (n:{nodeType})
            WITH n, toFloat(n.{propertyName}) AS value
            WHERE n.{propertyName} IS NOT NULL
            RETURN
                '{nodeType}' AS nodeType,
                '{propertyName}' AS propertyName,
                COUNT(value) AS count,
                COUNT(DISTINCT value) AS distinctCount,
                AVG(value) AS mean,
                MIN(value) AS min,
                MAX(value) AS max,
                COUNT(CASE WHEN n.{propertyName} IS NULL OR n.{propertyName} = '' OR toLower(trim(n.{propertyName})) IN ['nan', 'null', 'none', ''] THEN 1 ELSE NULL END) AS missingCount
        """
    
    try:
        stats_result = execute_read(driver, stats_query)
        if stats_result:
            stats_list.append(stats_result[0])
    except Exception as e:
        print(f"Error processing {nodeType}.{propertyName}: {e}")

# Convert to DataFrame
df = pd.DataFrame(stats_list, columns=['Node', 'Property', 'Count', 'Distinct Count', 
                                       'Mean', 'Min', 'Max', 'Missing Count'])                          # Convert the list to a DataFrame
df['Missing Count (%)'] = (df['Missing Count'] / df['Count']) * 100                                     # Calculate the percentage of missing values
df.set_index(['Node', 'Property'], inplace=True)                                                        # Set the index to Node and Property (Hierarchical Indexing)
df.fillna('-', inplace=True)                                                                            # Fill NaN values with 0
df = df.map(
    lambda x: '{:,.0f}'.format(x).replace(',', ' ') if isinstance(x, float) and not np.isnan(x) and not np.isinf(x) and x == int(x) else
              '{:,.2f}'.format(x).replace(',', ' ') if isinstance(x, float) and not np.isnan(x) and not np.isinf(x) else
              x
)                                                                                                                     # Format the DataFrame for better visualization
df 

## Time of Execution: 2m (1st query) + 07min 38s (2nd query) = 9m 38s

100%|██████████| 27/27 [08:04<00:00, 17.95s/it]


Count  Distinct Count          Mean   Min  \
Node      Property                                                    
BEERS     abv            372718             939          6.53  0.01   
          availability   417746              20             -     -   
          brewery_id     417746           16569     24 592.84     1   
          id             417746          358873    189 196.88     3   
          name           417746          298567             -     -   
          notes          417746           48313             -     -   
          retired        417746               2             -     -   
          state          417746              68             -     -   
BREWERIES id             100694           50347     27 870.51     1   
          name           100694           45245             -     -   
          notes          100694            3271             -     -   
          state          100694              68             -     -   
          types          100694              30             -     -   
CITIES    name            23330           11665             -     -   
COUNTRIES name              400             200             -     -   
REVIEWS   beer_id       2549252          189645     77 459.01     3   
          date          2549252            6379             -     -   
          feel          1484819              17          3.89     1   
          id            2549252         2546141  4 517 442.86     1   
          look          1484819              17          3.95     1   
          overall       1484819              17          3.92     1   
          score         2549252             401          3.89     1   
          smell         1484819              17          3.89     1   
          taste         1484819              17          3.92     1   
          text          2549252          814333             -     -   
STYLE     name              113             113             -     -   
USER      name           123935          123935             -     -   

                              Max  Missing Count Missing Count (%)  
Node      Property                                                  
BEERS     abv                 100          45028             12.08  
          availability          -              0                 0  
          brewery_id       54 144              0                 0  
          id              374 406              0                 0  
          name                  -              0                 0  
          notes                 -             55              0.01  
          retired               -              0                 0  
          state                 -          70831             16.96  
BREWERIES id               54 156              0                 0  
          name                  -              0                 0  
          notes                 -            170              0.17  
          state                 -          22542             22.39  
          types                 -              0                 0  
CITIES    name                  -              2              0.01  
COUNTRIES name                  -              2              0.50  
REVIEWS   beer_id         373 128              0                 0  
          date                  -             19              0.00  
          feel                  5        1064433             71.69  
          id            9 073 127              0                 0  
          look                  5        1064433             71.69  
          overall               5        1064433             71.69  
          score                 5              0                 0  
          smell                 5        1064433             71.69  
          taste                 5        1064433             71.69  
          text                  -             19              0.00  
STYLE     name                  -              1              0.88  
USER      name                  - 

<div class="alert alert-block alert-info" style="color: white; background-color: rgb(110, 3, 3); border: 0px; -moz-border-radius: 10px; -webkit-border-radius: 10px;">

#### **🖼️ Statistical Analysis of the Database**

</div>

**BEERS**
- **`abv`**: According to [Hops & Hopes (2025)](https://www.hopsandhopes.nl/en/beers/snake-venom) the world's strongest beer is **Snake Venom** with **67.5% ABV**. We can see that the **maximum** value in the database is **100**, so we will need to verify the beers with more than **67.5% ABV**.
- **`availability`** have **$20$** different values. We will check what are the possible values.
- **`brewery_id`**: In total we have **$417\;746$** brewery_ids, but **$16\;569$** distinct brewery_ids. It is possible that some breweries have more than one beer.
- **`id`**: In total we have **$417\;746$** ids, but **$358\;873$** distinct ids. We need to check the duplicates.
- **`name`**: In total we have **$417\;746$** names, but **$298\;567$** distinct names. It is possible that some beers have the same name.
- **`notes`**: In total we have **$417\;746$** notes, but **$48\;313$** distinct notes. Same as the **`name`** property, it is possible that some beers have the same notes.
- **`retired`** have **$2$** different values - **`True`** and **`False`**.
- **`state`** have **$68$** different values. We will check what are the possible values.

**BREWERIES**

- **`id`**: In total we have **$100\;694$** ids, but **$50\;347$** distinct ids. As in **`BEERS`**, it is possible that some breweries have more than one beer.
- **`name`**: In total we have **$100\;694$** names, but **$45\;245$** distinct names. We need to check the duplicates.
- **`notes`**: In total we have **$100\;694$** notes, but **$3\;271$** distinct notes. Same as the **`name`** property, it is possible that some breweries have the same notes.
- **`state`** have **$68$** different values (the same as in **`BEERS`**). We will check what are the possible values.
- **`types`** have **$30$** different values. We will check what are the possible values.

**CITIES**

- **`name`**: In total we have **$23\;330$** names, but **$11\;665$** distinct names. We need to check the duplicates.

**COUNTRIES**

- **`name`**: In total we have **$400$** names, but **$200$** distinct names. We need to check the duplicates.

**REVIEWS**

- **`beer_id`**: In total we have **$2\;549\;252$** beer_ids, but **$189\;645$** distinct beer_ids. It is possible that some beers have more than one review.
- **`date`**: We need to convert this property to **`date`** format.
- **`feel`**, **`look`**, **`overall`**, **`smell`**, **`taste`** and **`score`**: We need to convert these properties to **`float`** format, but we can see that the values are already clean, because the **`Min`** and **`Max`** values are between **$1$** and **$5$**.
- **`id`**: In total we have **$2\;549\;252$** ids, but **$2\;546\;141$** distinct ids. We need to check the duplicates.
- **`text`**: In total we have **$2\;549\;252$** texts, but **$814\;333$** distinct texts. It is possible that some reviews have the same text.

**STYLE**

- **`name`**: In total we have **$113$** names and distinct names, so we don't need to check the duplicates.

**USER**

- **`name`**: In total we have **$123\;935$** names and distinct names, so we don't need to check the duplicates.

---

## **2. Adjust the database**

- Adjust the database and mention reasoning behind: 
    - Clean errors, 
    - Remove redundancies, 
    - Adjust schema as necessary
    - Visualize the final version of database schema. [4 points]

In [19]:
# 2. Adjust the database and mention reasoning behind: 
#    e.g. clean errors, remove redundancies, adjust schema as necessary. 
# Visualize the final version of database schema. [4 points]

### **2.1. 🔢🔠 Adjust the data types**

In [21]:
# 2.1. Convert BEERS.brewery_id, BEERS.id, BREWERIES.id, REVIEWS.beer_id, REVIEWS.id to integer
# Convert BEERS.brewery_id and BEERS.id to integer
query = """
    MATCH (n:BEERS)
    SET n.brewery_id = toInteger(n.brewery_id)
    SET n.id = toInteger(n.id);
"""
result = execute_write(driver, query)
pprint(result)

# Convert BREWERIES.id to integer
query = """
    MATCH (n:BREWERIES)
    SET n.id = toInteger(n.id);
"""
result = execute_write(driver, query)
pprint(result)

# # Convert REVIEWS.beer_id and REVIEWS.id to integer
# query = """
#     MATCH (n:REVIEWS)
#     SET n.beer_id = toInteger(n.beer_id)
#     SET n.id = toInteger(n.id)
# """
# result = execute_write(driver, query)
# pprint(result)

# {code: Neo.DatabaseError.Statement.ExecutionFailed} {message: Java heap space}

[]
[]


When we run the previous query we raise an error

```
DatabaseError                             Traceback (most recent call last)
....
DatabaseError: {code: Neo.DatabaseError.Statement.ExecutionFailed} {message: Java heap space}


Transaction failed and will be retried in 0.8927118089735324s (There is not enough memory to perform the current task. Please try increasing 'dbms.memory.heap.max_size' in the neo4j configuration (normally in 'conf/neo4j.conf' or, if you are using Neo4j Desktop, found through the user interface) or if you are running an embedded installation increase the heap by using '-Xmx' command line flag, and then restart the database.)
```

> To solve this problem we try to **increase the heap size** of the database, but we still get the same error. So after some research we create a function with help of the **`AI`** to ***Batch Processing*** the writing in the database.


In [22]:
# Source: X. (2025). Grok 3 Beta — The Age of Reasoning Agents (Mar 07 version)[Large Language Model]. X.ai. https://x.ai/blog/grok-3
#         Mistral. (2025). Le Chat - Mistral AI (Mar 07 version)[Large Language Model]. Mistral.ai. https://chat.mistral.ai/

def execute_batch_write(driver, node_label, operation_clause, total_nodes, batch_size=100_000, desc="Processing batches"):
    """
    Process write operations in batches for a given node label.

    Parameters:
    - driver: Neo4j driver instance.
    - node_label: String, the label of nodes to process (e.g., 'REVIEWS', 'BEERS').
    - operation_clause: String, the Cypher clause to execute on matched nodes (e.g., 'SET n.prop = value').
    - total_nodes: Integer, total number of nodes to process.
    - batch_size: Integer, number of nodes per batch (default: 50,000).
    - desc: String, description for tqdm progress bar (default: 'Processing batches').

    Returns:
    - None (prints progress and errors if any).
    """
    for offset in tqdm(range(0, total_nodes, batch_size), desc=desc):
        print(f"Processing batch from offset {offset} to {offset + batch_size}")
        
        # Base query template with SKIP and LIMIT integrated correctly
        batch_query = f"""
            MATCH (n:{node_label})
            WITH n
            SKIP {offset} LIMIT {batch_size}
            {operation_clause}
            RETURN COUNT(*)
        """
        # print(batch_query.strip())  # Show the query for debugging
        
        try:
            execute_write(driver, batch_query)
        except Exception as e:
            print(f"Error at offset {offset}: {e}")
            break

In [23]:
# Total node counts from your exploration (1.5)
total_reviews = 2_549_252
operation_clause = "SET n.beer_id = toInteger(n.beer_id), n.id = toInteger(n.id)"
execute_batch_write(driver, "REVIEWS", operation_clause, total_reviews, desc="Converting REVIEWS ids to integer")

Converting REVIEWS ids to integer:   0%|          | 0/26 [00:00<?, ?it/s]

Processing batch from offset 0 to 100000


Converting REVIEWS ids to integer:   4%|▍         | 1/26 [00:11<04:39, 11.19s/it]

Processing batch from offset 100000 to 200000


Converting REVIEWS ids to integer:   8%|▊         | 2/26 [00:16<02:58,  7.45s/it]

...

Converting REVIEWS ids to integer:  92%|█████████▏| 24/26 [04:36<00:19,  9.86s/it]

Processing batch from offset 2400000 to 2500000


Converting REVIEWS ids to integer:  96%|█████████▌| 25/26 [04:43<00:09,  9.12s/it]

Processing batch from offset 2500000 to 2600000


Converting REVIEWS ids to integer: 100%|██████████| 26/26 [05:01<00:00, 11.60s/it]


In [24]:
# Verify the changes
query = """
    MATCH (n:BEERS)
    RETURN n.brewery_id, n.id
    LIMIT 5
"""
result = execute_read(driver, query)
pprint(result)

query = """
    MATCH (n:BREWERIES)
    RETURN n.id
    LIMIT 5
"""
result = execute_read(driver, query)
pprint(result)

query = """
    MATCH (n:REVIEWS)
    RETURN n.beer_id, n.id
    LIMIT 5
"""
result = execute_read(driver, query)
pprint(result)

[<Record n.brewery_id=2199 n.id=202522>,
 <Record n.brewery_id=18604 n.id=82352>,
 <Record n.brewery_id=44306 n.id=214879>,
 <Record n.brewery_id=4378 n.id=320009>,
 <Record n.brewery_id=44617 n.id=246438>]
[<Record n.id=19730>,
 <Record n.id=32541>,
 <Record n.id=44736>,
 <Record n.id=23372>,
 <Record n.id=35328>]
[<Record n.beer_id=125646 n.id=1>,
 <Record n.beer_id=125646 n.id=2>,
 <Record n.beer_id=125646 n.id=3>,
 <Record n.beer_id=125646 n.id=4>,
 <Record n.beer_id=125646 n.id=6>]


In [25]:
# 2.1. Convert BEERS.abv, REVIEWS.look, REVIEWS.overall, REVIEWS.score, REVIEWS.smell, REVIEWS.taste to float
query = """
    MATCH (n:BEERS)
    SET n.abv = toFloat(n.abv);
"""
result = execute_write(driver, query)
pprint(result)

# query = """
#     MATCH (n:REVIEWS)
#     SET n.feel = toFloat(n.feel)
#     SET n.look = toFloat(n.look)
#     SET n.overall = toFloat(n.overall)
#     SET n.score = toFloat(n.score)
#     SET n.smell = toFloat(n.smell)
#     SET n.taste = toFloat(n.taste)
# """
# result = execute_write(driver, query)
# pprint(result)                                      # Original query failed due to Java heap space error

# Total node counts from your exploration (1.5)
total_reviews = 2_549_252

# Convert REVIEWS properties to float in batches
operation_clause = """
    SET n.feel = toFloat(n.feel),
        n.look = toFloat(n.look), 
        n.overall = toFloat(n.overall), 
        n.score = toFloat(n.score), 
        n.smell = toFloat(n.smell), 
        n.taste = toFloat(n.taste)
"""
execute_batch_write(driver, "REVIEWS", operation_clause, total_reviews, batch_size=250_000,  desc="Converting REVIEWS properties to float")

[]


Converting REVIEWS properties to float:   0%|          | 0/11 [00:00<?, ?it/s]

Processing batch from offset 0 to 250000


Converting REVIEWS properties to float:   9%|▉         | 1/11 [00:36<06:07, 36.73s/it]

Processing batch from offset 250000 to 500000


...

Converting REVIEWS properties to float:  82%|████████▏ | 9/11 [04:36<00:56, 28.47s/it]

Processing batch from offset 2250000 to 2500000


Converting REVIEWS properties to float:  91%|█████████ | 10/11 [05:11<00:30, 30.31s/it]

Processing batch from offset 2500000 to 2750000


Converting REVIEWS properties to float: 100%|██████████| 11/11 [05:15<00:00, 28.69s/it]


In [26]:
# Verify the changes
query = """
    MATCH (n:BEERS)
    RETURN n.abv
    LIMIT 5
"""
result = execute_read(driver, query)
pprint(result)

query = """
    MATCH (n:REVIEWS)
    RETURN n.look, n.overall, n.score, n.smell, n.taste
    LIMIT 5
"""
result = execute_read(driver, query)
pprint(result)

[<Record n.abv=7.3>,
 <Record n.abv=10.4>,
 <Record n.abv=4.0>,
 <Record n.abv=8.7>,
 <Record n.abv=5.1>]
[<Record n.look=4.5 n.overall=4.5 n.score=4.5 n.smell=4.5 n.taste=4.5>,
 <Record n.look=4.75 n.overall=4.75 n.score=4.75 n.smell=4.75 n.taste=4.75>,
 <Record n.look=4.75 n.overall=4.5 n.score=4.58 n.smell=4.75 n.taste=4.5>,
 <Record n.look=4.25 n.overall=4.25 n.score=4.31 n.smell=4.5 n.taste=4.25>,
 <Record n.look=4.75 n.overall=4.75 n.score=4.69 n.smell=4.5 n.taste=4.75>]


In [27]:
# 2.1. Convert REVIEWS.date to date format
#      Source: https://neo4j.com/developer/kb/neo4j-string-to-date/

# Confirm that the date format is in the correct format (YYYY-MM-DD)
query = """
    MATCH (n:REVIEWS)
    WITH n.date AS date
    RETURN DISTINCT date
    LIMIT 5
"""
result = execute_read(driver, query)
pprint(result)

[<Record date='2017-12-21'>,
 <Record date='2017-12-20'>,
 <Record date='2017-12-19'>,
 <Record date='2017-12-10'>,
 <Record date='2017-12-09'>]


In [28]:
# Total node counts from your exploration (1.5)
total_reviews = 2_549_252

# Convert REVIEWS.date to date format in batches
operation_clause = """
    SET n.date = date(n.date)
"""
execute_batch_write(driver, "REVIEWS", operation_clause, total_reviews, batch_size=250_000, desc="Converting REVIEWS date to date format")

Converting REVIEWS date to date format:   0%|          | 0/11 [00:00<?, ?it/s]

Processing batch from offset 0 to 250000


Converting REVIEWS date to date format:   9%|▉         | 1/11 [00:07<01:11,  7.10s/it]

Processing batch from offset 250000 to 500000


Converting REVIEWS date to date format:  18%|█▊        | 2/11 [00:12<00:53,  5.98s/it]

...

Converting REVIEWS date to date format:  82%|████████▏ | 9/11 [01:14<00:18,  9.18s/it]

Processing batch from offset 2250000 to 2500000


Converting REVIEWS date to date format:  91%|█████████ | 10/11 [01:22<00:08,  8.65s/it]

Processing batch from offset 2500000 to 2750000


Converting REVIEWS date to date format: 100%|██████████| 11/11 [01:23<00:00,  7.63s/it]


In [29]:
# Verify the changes
query = """
    MATCH (n:REVIEWS)
    RETURN n.date
    LIMIT 5
"""
result = execute_read(driver, query)
pprint(result)

[<Record n.date=neo4j.time.Date(2017, 12, 21)>,
 <Record n.date=neo4j.time.Date(2017, 12, 21)>,
 <Record n.date=neo4j.time.Date(2017, 12, 20)>,
 <Record n.date=neo4j.time.Date(2017, 12, 20)>,
 <Record n.date=neo4j.time.Date(2017, 12, 19)>]


In [30]:
# Convert BEERS.retired to boolean format
query = """
    MATCH (r:BEERS)
    SET r.retired = CASE 
            WHEN toLower(trim(r.retired)) = 't' THEN true 
            WHEN toLower(trim(r.retired)) = 'f' THEN false 
            ELSE r.retired
        END 
"""
result = execute_write(driver, query)
pprint(result)

[]


In [31]:
# Verify the changes
query = """
    MATCH (n:BEERS)
    RETURN n.retired
    LIMIT 5
"""
result = execute_read(driver, query)
pprint(result)

[<Record n.retired=False>,
 <Record n.retired=False>,
 <Record n.retired=True>,
 <Record n.retired=False>,
 <Record n.retired=False>]


In [32]:
# Re-run the statistics query (1.7 & 1.12) to get the updated schema after the data type conversions

# Get all properties for each node label
query = """
    CALL db.schema.nodeTypeProperties() 
    YIELD nodeType, propertyName, propertyTypes, mandatory 
    WITH nodeType, propertyName, propertyTypes, mandatory 
    ORDER BY nodeType, propertyName 
    RETURN nodeType, propertyName, propertyTypes, mandatory
"""
properties_result = execute_read(driver, query)

print("Properties and types for each node label:")
for rec in properties_result:
    print(f"Node Label: {rec['nodeType']:<15} | Property: {rec['propertyName']:<15} | Type(s): {', '.join(rec['propertyTypes']):<20} | Mandatory: {rec['mandatory']}")

Properties and types for each node label:
Node Label: :`BEERS`        | Property: abv             | Type(s): Double               | Mandatory: False
Node Label: :`BEERS`        | Property: availability    | Type(s): String               | Mandatory: True
Node Label: :`BEERS`        | Property: brewery_id      | Type(s): Long                 | Mandatory: True
Node Label: :`BEERS`        | Property: id              | Type(s): Long                 | Mandatory: True
Node Label: :`BEERS`        | Property: name            | Type(s): String               | Mandatory: True
Node Label: :`BEERS`        | Property: notes           | Type(s): String               | Mandatory: True
Node Label: :`BEERS`        | Property: retired         | Type(s): Boolean              | Mandatory: True
Node Label: :`BEERS`        | Property: state           | Type(s): String               | Mandatory: True
Node Label: :`BREWERIES`    | Property: id              | Type(s): Long                 | Mandatory: True
Nod

In [33]:
print("\n\nSummary statistics of each property for each node label in the database:")

# Initialize a list to store statistics
stats_list = []

# Iterate over each property and calculate statistics
for record in tqdm(properties_result):
    nodeType = record['nodeType'].replace('`', '').replace(':', '')  # Remove backticks and ':'
    propertyName = record['propertyName']
    
    # Construct query for each property
    if propertyName.lower() in ['name', 'notes', 'state', 'availability', 'types', 'text']:
        # String properties
        stats_query = f"""
            MATCH (n:{nodeType})
            RETURN
                '{nodeType}' AS nodeType,
                '{propertyName}' AS propertyName,
                COUNT(n.{propertyName}) AS count,
                COUNT(DISTINCT n.{propertyName}) AS distinctCount,
                '-' AS mean,
                '-' AS min,
                '-' AS max,
                COUNT(CASE WHEN n.{propertyName} IS NULL OR n.{propertyName} = '' OR toLower(trim(n.{propertyName})) IN ['nan', 'null', 'none', ' '] THEN 1 ELSE NULL END) AS missingCount
        """
    elif propertyName.lower() in ['retired', 'date']:
        # Boolean & Date properties
        stats_query = f"""
            MATCH (n:{nodeType})
            RETURN
                '{nodeType}' AS nodeType,
                '{propertyName}' AS propertyName,
                COUNT(n.{propertyName}) AS count,
                COUNT(DISTINCT n.{propertyName}) AS distinctCount,
                '-' AS mean,
                '-' AS min,
                '-' AS max,
                COUNT(CASE WHEN n.{propertyName} IS NULL THEN 1 ELSE NULL END) AS missingCount
        """
    else:
        # Numeric properties
        stats_query = f"""
            MATCH (n:{nodeType})
            WITH n, n.{propertyName} AS value
            RETURN
                '{nodeType}' AS nodeType,
                '{propertyName}' AS propertyName,
                COUNT(value) AS count,
                COUNT(DISTINCT value) AS distinctCount,
                AVG(value) AS mean,
                MIN(value) AS min,
                MAX(value) AS max,
                COUNT(CASE WHEN value IS NULL THEN 1 ELSE NULL END) AS missingCount
        """
    
    try:
        stats_result = execute_read(driver, stats_query)
        if stats_result:
            stats_list.append(stats_result[0])
    except Exception as e:
        print(f"Error processing {nodeType}.{propertyName}: {e}")

# Convert to DataFrame
df = pd.DataFrame(stats_list, columns=['Node', 'Property', 'Count', 'Distinct Count', 
                                       'Mean', 'Min', 'Max', 'Missing Count'])                          # Convert the list to a DataFrame
df['Missing Count (%)'] = (df['Missing Count'] / df['Count']) * 100                                     # Calculate the percentage of missing values
df.set_index(['Node', 'Property'], inplace=True)                                                        # Set the index to Node and Property (Hierarchical Indexing)
df.fillna('-', inplace=True)                                                                            # Fill NaN values with 0
df = df.map(
    lambda x: '{:,.0f}'.format(x).replace(',', ' ') if isinstance(x, float) and not np.isnan(x) and not np.isinf(x) and x == int(x) else
              '{:,.2f}'.format(x).replace(',', ' ') if isinstance(x, float) and not np.isnan(x) and not np.isinf(x) else
              x
)                                                                                                                     # Format the DataFrame for better visualization
df 



Summary statistics of each property for each node label in the database:


100%|██████████| 27/27 [07:22<00:00, 16.41s/it]


Count  Distinct Count          Mean   Min      Max  \
Node      Property                                                             
BEERS     abv            372718             939          6.53  0.01      100   
          availability   417746              20             -     -        -   
          brewery_id     417746           16569     24 592.84     1    54144   
          id             417746          358873    189 196.88     3   374406   
          name           417746          298567             -     -        -   
          notes          417746           48313             -     -        -   
          retired        417746               2             -     -        -   
          state          417746              68             -     -        -   
BREWERIES id             100694           50347     27 870.51     1    54156   
          name           100694           45245             -     -        -   
          notes          100694            3271             -     -        -   
          state          100694              68             -     -        -   
          types          100694              30             -     -        -   
CITIES    name            23330           11665             -     -        -   
COUNTRIES name              400             200             -     -        -   
REVIEWS   beer_id       2549252          189645     77 459.01     3   373128   
          date          2549252            6379             -     -        -   
          feel          1484819              17          3.89     1        5   
          id            2549252         2546141  4 517 442.86     1  9073127   
          look          1484819              17          3.95     1        5   
          overall       1484819              17          3.92     1        5   
          score         2549252             401          3.89     1        5   
          smell         1484819              17          3.89     1        5   
          taste         1484819              17          3.92     1        5   
          text          2549252          814333             -     -        -   
STYLE     name              113             113             -     -        -   
USER      name           123935          123935             -     -        -   

                        Missing Count Missing Count (%)  
Node      Property                                       
BEERS     abv                   45028             12.08  
          availability              0                 0  
          brewery_id                0                 0  
          id                        0                 0  
          name                      0                 0  
          notes                    55              0.01  
          retired                   0                 0  
          state                 70831             16.96  
BREWERIES id                        0                 0  
          name                      0                 0  
          notes                   170              0.17  
          state                 22542             22.39  
          types                     0                 0  
CITIES    name                      2              0.01  
COUNTRIES name                      2              0.50  
REVIEWS   beer_id                  19              0.00  
          date                     19              0.00  
          feel                1064452             71.69  
          id                       19              0.00  
          look                1064452             71.69  
          overall             1064452             71.69  
          score                    19              0.00  
          smell               1064452             71.69  
          taste               1064452             71.69  
          text                     19              0.00  
STYLE     name                      1              0.88  
USER      name                      1              0.00

### **2.2. 🔄️ Find Duplicate Nodes**

In [34]:
# 2.2. Find duplicate nodes if any (BEERS)
query = """
    // Find duplicate BEERS nodes
    MATCH (b:BEERS)
    WITH b.id AS beerId, count(b) AS count
    WHERE count > 1
    RETURN beerId, count
    LIMIT 10
"""
result = execute_read(driver, query)
pprint(result)

query = """
    // Find the total number of duplicate BEERS nodes
    MATCH (b:BEERS)
    WITH b.id AS beerId, count(b) AS count
    WHERE count > 1
    RETURN count(beerId) AS totalDuplicates
"""
result = execute_read(driver, query)
pprint(result)

[<Record beerId=214879 count=2>,
 <Record beerId=8036 count=2>,
 <Record beerId=265827 count=2>,
 <Record beerId=110318 count=2>,
 <Record beerId=138971 count=2>,
 <Record beerId=199068 count=2>,
 <Record beerId=80010 count=2>,
 <Record beerId=55175 count=2>,
 <Record beerId=55 count=2>,
 <Record beerId=354484 count=2>]
[<Record totalDuplicates=58873>]


> In total we have $58873$ duplicate **`BEERS`** ($\frac{58\;873}{417\;746} \approx 14.1\%$)

In [35]:
# Verify if the duplicate nodes have same relationships
query = """
    // Verify if the duplicate nodes have the same relationships
    MATCH (b:BEERS)
    WITH b.id AS beerId, count(b) AS count, collect(b) AS nodes
    WHERE count > 1
    WITH beerId, nodes
    UNWIND nodes AS node1
    UNWIND nodes AS node2
    WITH beerId, node1, node2
    WHERE id(node1) <> id(node2)
    // Collect relationships for each node
    OPTIONAL MATCH (node1)-[r1]-()
    OPTIONAL MATCH (node2)-[r2]-()
    WITH beerId, node1, node2, collect(r1) AS rels1, collect(r2) AS rels2
    // Compare relationships
    WHERE rels1 <> rels2
    RETURN beerId, id(node1) AS id1, id(node2) AS id2, rels1, rels2
    LIMIT 2
"""
result = execute_read(driver, query)
pprint(result)

query = """
    // Verify if the duplicate nodes have the same relationships
    MATCH (b:BEERS)
    WITH b.id AS beerId, count(b) AS count, collect(b) AS nodes
    WHERE count > 1
    WITH beerId, nodes
    UNWIND nodes AS node1
    UNWIND nodes AS node2
    WITH beerId, node1, node2
    WHERE id(node1) <> id(node2)
    // Collect relationships for each node
    OPTIONAL MATCH (node1)-[r1]-()
    OPTIONAL MATCH (node2)-[r2]-()
    WITH beerId, node1, node2, collect(r1) AS rels1, collect(r2) AS rels2
    // Compare relationships
    WHERE rels1 <> rels2
    // Count how many cases
    RETURN count(beerId) AS totalCases
"""
result = execute_read(driver, query)
pprint(result)

[<Record beerId=214879 id1=62214 id2=9721475 rels1=[<Relationship element_id='9494215' nodes=(<Node element_id='62214' labels=frozenset() properties={}>, <Node element_id='9494215' labels=frozenset() properties={}>) type='HAS_STYLE' properties={}>, <Relationship element_id='9072917' nodes=(<Node element_id='13515' labels=frozenset() properties={}>, <Node element_id='62214' labels=frozenset() properties={}>) type='BREWED' properties={}>] rels2=[]>,
 <Record beerId=214879 id1=9721475 id2=62214 rels1=[] rels2=[<Relationship element_id='9494215' nodes=(<Node element_id='62214' labels=frozenset() properties={}>, <Node element_id='9494215' labels=frozenset() properties={}>) type='HAS_STYLE' properties={}>, <Relationship element_id='9072917' nodes=(<Node element_id='13515' labels=frozenset() properties={}>, <Node element_id='62214' labels=frozenset() properties={}>) type='BREWED' properties={}>]>]
[<Record totalCases=117746>]


In [36]:
# 2.2 Find duplicate nodes if any (BREWERIES)
query = """
    // Find duplicate BREWERIES nodes
    MATCH (b:BREWERIES)
    WITH b.id AS breweryId, b.name AS breweryName, count(b) AS count
    WHERE count > 1
    RETURN breweryId, breweryName, count
    LIMIT 10
"""
result = execute_read(driver, query)
pprint(result)

query = """
    // Find the total number of duplicate BREWERIES nodes
    MATCH (b:BREWERIES)
    WITH b.id AS breweryId, count(b) AS count
    WHERE count > 1
    RETURN count(breweryId) AS totalDuplicates
"""
result = execute_read(driver, query)
pprint(result)

[<Record breweryId=19730 breweryName='Brouwerij Danny' count=2>,
 <Record breweryId=32541 breweryName='Coachella Valley Brewing Co' count=2>,
 <Record breweryId=44736 breweryName="Beef 'O' Brady's" count=2>,
 <Record breweryId=23372 breweryName='Broadway Wine Merchant' count=2>,
 <Record breweryId=35328 breweryName='Brighton Beer Dispensary (DUPLICATE)' count=2>,
 <Record breweryId=31561 breweryName="Teddy's Tavern" count=2>,
 <Record breweryId=35975 breweryName='Modus Operandi Brewing Co.' count=2>,
 <Record breweryId=5618 breweryName='Hops! Beer Restaurant & Pizza' count=2>,
 <Record breweryId=30916 breweryName="Kelly's Cellars" count=2>,
 <Record breweryId=41278 breweryName='The Other End' count=2>]
[<Record totalDuplicates=50347>]


> In total we have $50; 347$ duplicate **`BREWERIES`** ($\frac{50\;347}{100\;694} \approx 50\%$)

In [37]:
# Verify if the duplicate nodes have same relationships
query = """
    // Verify if the duplicate nodes have the same relationships
    MATCH (b:BREWERIES)
    WITH b.id AS breweryId, count(b) AS count, collect(b) AS nodes
    WHERE count > 1
    WITH breweryId, nodes
    UNWIND nodes AS node1
    UNWIND nodes AS node2
    WITH breweryId, node1, node2
    WHERE id(node1) <> id(node2)
    // Collect relationships for each node
    OPTIONAL MATCH (node1)-[r1]-()
    OPTIONAL MATCH (node2)-[r2]-()
    WITH breweryId, node1, node2, collect(r1) AS rels1, collect(r2) AS rels2
    // Compare relationships
    WHERE rels1 <> rels2
    RETURN breweryId, id(node1) AS id1, id(node2) AS id2, rels1, rels2
    LIMIT 2
"""
result = execute_read(driver, query)
pprint(result)

query = """
    // Verify if the duplicate nodes have the same relationships
    MATCH (b:BREWERIES)
    WITH b.id AS breweryId, count(b) AS count, collect(b) AS nodes
    WHERE count > 1
    WITH breweryId, nodes
    UNWIND nodes AS node1
    UNWIND nodes AS node2
    WITH breweryId, node1, node2
    WHERE id(node1) <> id(node2)
    // Collect relationships for each node
    OPTIONAL MATCH (node1)-[r1]-()
    OPTIONAL MATCH (node2)-[r2]-()
    WITH breweryId, node1, node2, collect(r1) AS rels1, collect(r2) AS rels2
    // Compare relationships
    WHERE rels1 <> rels2
    // Count how many cases
    RETURN count(breweryId) AS totalCases
"""
result = execute_read(driver, query)
pprint(result)

[<Record breweryId=19730 id1=11865 id2=9671126 rels1=[<Relationship element_id='9431789' nodes=(<Node element_id='11865' labels=frozenset() properties={}>, <Node element_id='200' labels=frozenset() properties={}>) type='IN' properties={}>, <Relationship element_id='9254060' nodes=(<Node element_id='11865' labels=frozenset() properties={}>, <Node element_id='243356' labels=frozenset() properties={}>) type='BREWED' properties={}>, <Relationship element_id='9165745' nodes=(<Node element_id='11865' labels=frozenset() properties={}>, <Node element_id='155042' labels=frozenset() properties={}>) type='BREWED' properties={}>, <Relationship element_id='9108729' nodes=(<Node element_id='11865' labels=frozenset() properties={}>, <Node element_id='98026' labels=frozenset() properties={}>) type='BREWED' properties={}>] rels2=[]>,
 <Record breweryId=19730 id1=9671126 id2=11865 rels1=[] rels2=[<Relationship element_id='9431789' nodes=(<Node element_id='11865' labels=frozenset() properties={}>, <Node 

In [38]:
# Test the code below
# 2.2.* Find a BREWERIES node with name "Brouwerij Danny" and check if there are duplicates
query = """
    MATCH (b:BREWERIES {name: "Brouwerij Danny"})
    RETURN id(b), b{.*}
"""
result = execute_read(driver, query)
pprint(result)

[<Record id(b)=11865 b={'types': 'Brewery', 'notes': 'No notes at this time.', 'name': 'Brouwerij Danny', 'state': 'nan', 'id': 19730}>,
 <Record id(b)=9671126 b={'types': 'Brewery', 'notes': 'No notes at this time.', 'name': 'Brouwerij Danny', 'state': 'nan', 'id': 19730}>]


In [39]:
# 2.2. Find duplicate nodes if any (COUNTRIES)
query = """
    // Find duplicate COUNTRIES nodes
    MATCH (c:COUNTRIES)
    WITH c.name AS countryName, count(c) AS count
    WHERE count > 1
    RETURN countryName, count
    LIMIT 10
"""
result = execute_read(driver, query)
pprint(result)

query = """
    // Find the total number of duplicate COUNTRIES nodes
    MATCH (c:COUNTRIES)
    WITH c.name AS countryName, count(c) AS count
    WHERE count > 1
    RETURN count(countryName) AS totalDuplicates
"""
result = execute_read(driver, query)
pprint(result)

[<Record countryName='BE' count=2>,
 <Record countryName='US' count=2>,
 <Record countryName='GB' count=2>,
 <Record countryName='AU' count=2>,
 <Record countryName='IT' count=2>,
 <Record countryName='CA' count=2>,
 <Record countryName='GR' count=2>,
 <Record countryName='FR' count=2>,
 <Record countryName='AT' count=2>,
 <Record countryName='ES' count=2>]
[<Record totalDuplicates=200>]


> In total we have $200$ duplicate **`COUNTRIES`** ($\frac{200}{400} = 50\%$)

In [40]:
# Adicional checking: Making sure if the ISO codes are real/valid
query = """
        MATCH (c:COUNTRIES)
        WHERE NOT c.code IN ['AD', 'AE', 'AF', 'AG', 'AI', 'AL', 'AM', 'AO', 'AQ', 'AR', 'AS', 'AT', 'AU', 'AW', 
            'AX', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BL', 'BM', 'BN', 'BO', 'BQ', 'BR', 
            'BS', 'BT', 'BV', 'BW', 'BY', 'BZ', 'CA', 'CC', 'CD', 'CF', 'CG', 'CH', 'CI', 'CK', 'CL', 'CM', 'CN', 
            'CO', 'CR', 'CU', 'CV', 'CW', 'CX', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 
            'EH', 'ER', 'ES', 'ET', 'FI', 'FJ', 'FK', 'FM', 'FO', 'FR', 'GA', 'GB', 'GD', 'GE', 'GF', 'GG', 'GH', 
            'GI', 'GL', 'GM', 'GN', 'GP', 'GQ', 'GR', 'GS', 'GT', 'GU', 'GW', 'GY', 'HK', 'HM', 'HN', 'HR', 'HT', 
            'HU', 'ID', 'IE', 'IL', 'IM', 'IN', 'IO', 'IQ', 'IR', 'IS', 'IT', 'JE', 'JM', 'JO', 'JP', 'KE', 'KG', 
            'KH', 'KI', 'KM', 'KN', 'KP', 'KR', 'KW', 'KY', 'KZ', 'LA', 'LB', 'LC', 'LI', 'LK', 'LR', 'LS', 'LT', 
            'LU', 'LV', 'LY', 'MA', 'MC', 'MD', 'ME', 'MF', 'MG', 'MH', 'MK', 'ML', 'MM', 'MN', 'MO', 'MP', 'MQ', 
            'MR', 'MS', 'MT', 'MU', 'MV', 'MW', 'MX', 'MY', 'MZ', 'NA', 'NC', 'NE', 'NF', 'NG', 'NI', 'NL', 'NO', 
            'NP', 'NR', 'NU', 'NZ', 'OM', 'PA', 'PE', 'PF', 'PG', 'PH', 'PK', 'PL', 'PM', 'PN', 'PR', 'PS', 'PT', 
            'PW', 'PY', 'QA', 'RE', 'RO', 'RS', 'RU', 'RW', 'SA', 'SB', 'SC', 'SD', 'SE', 'SG', 'SH', 'SI', 'SJ', 
            'SK', 'SL', 'SM', 'SN', 'SO', 'SR', 'SS', 'ST', 'SV', 'SX', 'SY', 'SZ', 'TC', 'TD', 'TF', 'TG', 'TH', 
            'TJ', 'TK', 'TL', 'TM', 'TN', 'TO', 'TR', 'TT', 'TV', 'TW', 'TZ', 'UA', 'UG', 'UM', 'US', 'UY', 'UZ', 
            'VA', 'VC', 'VE', 'VG', 'VI', 'VN', 'VU', 'WF', 'WS', 'YE', 'YT', 'ZA', 'ZM', 'ZW']
        RETURN c.name
"""

result = execute_read(driver, query)

pprint(result)

[]


- The name used for countries is <a href="https://www.iban.com/country-codes" target="_blank">Alpha-2</a><br>
- With the previous query we can see that we all `COUNTRIE.name` are codes that exists and are valid.

In [41]:
# # Verify if the duplicate nodes have same relationships
# query = """
#     // Verify if the duplicate nodes have the same relationships
#     MATCH (c:COUNTRIES)
#     WITH c.name AS countryName, count(c) AS count, collect(c) AS nodes
#     WHERE count > 1
#     WITH countryName, nodes
#     UNWIND nodes AS node1
#     UNWIND nodes AS node2
#     WITH countryName, node1, node2
#     WHERE id(node1) <> id(node2)
#     // Collect relationships for each node
#     OPTIONAL MATCH (node1)-[r1]-()
#     OPTIONAL MATCH (node2)-[r2]-()
#     WITH countryName, node1, node2, collect(r1) AS rels1, collect(r2) AS rels2
#     // Compare relationships
#     WHERE rels1 <> rels2
#     RETURN countryName, id(node1) AS id1, id(node2) AS id2, rels1, rels2
#     LIMIT 1
# """
# result = execute_read(driver, query)
# pprint(result)                                                                        # Large Output

query = """
    // Verify if the duplicate nodes have the same relationships
    MATCH (c:COUNTRIES)
    WITH c.name AS countryName, count(c) AS count, collect(c) AS nodes
    WHERE count > 1
    WITH countryName, nodes
    UNWIND nodes AS node1
    UNWIND nodes AS node2
    WITH countryName, node1, node2
    WHERE id(node1) <> id(node2)
    // Collect relationships for each node
    OPTIONAL MATCH (node1)-[r1]-()
    OPTIONAL MATCH (node2)-[r2]-()
    WITH countryName, node1, node2, collect(r1) AS rels1, collect(r2) AS rels2
    // Compare relationships
    WHERE rels1 <> rels2
    // Count how many cases
    RETURN count(countryName) AS totalCases
"""
result = execute_read(driver, query)
pprint(result)

[<Record totalCases=400>]


In [42]:
# 2.2. Find duplicate nodes if any (CITIES)
query = """
    // Find duplicate CITIES nodes
    MATCH (c:CITIES)
    WITH c.name AS cityName, count(c) AS count
    WHERE count > 1
    RETURN cityName, count
    LIMIT 10
"""
result = execute_read(driver, query)
pprint(result)

query = """
    // Find the total number of duplicate CITIES nodes
    MATCH (c:CITIES)
    WITH c.name AS cityName, count(c) AS count
    WHERE count > 1
    RETURN count(cityName) AS totalDuplicates
"""
result = execute_read(driver, query)
pprint(result)

[<Record cityName='Erpe-Mere' count=2>,
 <Record cityName='Thousand Palms' count=2>,
 <Record cityName='Plant City' count=2>,
 <Record cityName='Oklahoma City' count=2>,
 <Record cityName='Brighton' count=2>,
 <Record cityName='Seattle' count=2>,
 <Record cityName='Mona Vale' count=2>,
 <Record cityName='Riccione (RN)' count=2>,
 <Record cityName='Belfast' count=2>,
 <Record cityName='Destin' count=2>]
[<Record totalDuplicates=11665>]


> In total we have $11; 665$ duplicate **`CITIES`** ($\frac{11\;665}{23\;330} = 50\%$)

In [43]:
# Verify if the duplicate nodes have same relationships
query = """
    // Verify if the duplicate nodes have the same relationships
    MATCH (c:CITIES)
    WITH c.name AS cityName, count(c) AS count, collect(c) AS nodes
    WHERE count > 1
    WITH cityName, nodes
    UNWIND nodes AS node1
    UNWIND nodes AS node2
    WITH cityName, node1, node2
    WHERE id(node1) <> id(node2)
    // Collect relationships for each node
    OPTIONAL MATCH (node1)-[r1]-()
    OPTIONAL MATCH (node2)-[r2]-()
    WITH cityName, node1, node2, collect(r1) AS rels1, collect(r2) AS rels2
    // Compare relationships
    WHERE rels1 <> rels2
    RETURN cityName, id(node1) AS id1, id(node2) AS id2, rels1, rels2
    LIMIT 2
"""
result = execute_read(driver, query)
pprint(result)

query = """
    // Verify if the duplicate nodes have the same relationships
    MATCH (c:CITIES)
    WITH c.name AS cityName, count(c) AS count, collect(c) AS nodes
    WHERE count > 1
    WITH cityName, nodes
    UNWIND nodes AS node1
    UNWIND nodes AS node2
    WITH cityName, node1, node2
    WHERE id(node1) <> id(node2)
    // Collect relationships for each node
    OPTIONAL MATCH (node1)-[r1]-()
    OPTIONAL MATCH (node2)-[r2]-()
    WITH cityName, node1, node2, collect(r1) AS rels1, collect(r2) AS rels2
    // Compare relationships
    WHERE rels1 <> rels2
    // Count how many cases
    RETURN count(cityName) AS totalCases
"""
result = execute_read(driver, query)
pprint(result)

[<Record cityName='Erpe-Mere' id1=200 id2=9659461 rels1=[<Relationship element_id='9482136' nodes=(<Node element_id='200' labels=frozenset() properties={}>, <Node element_id='0' labels=frozenset() properties={}>) type='IN' properties={}>, <Relationship element_id='9458134' nodes=(<Node element_id='38210' labels=frozenset() properties={}>, <Node element_id='200' labels=frozenset() properties={}>) type='IN' properties={}>, <Relationship element_id='9431789' nodes=(<Node element_id='11865' labels=frozenset() properties={}>, <Node element_id='200' labels=frozenset() properties={}>) type='IN' properties={}>] rels2=[]>,
 <Record cityName='Erpe-Mere' id1=9659461 id2=200 rels1=[] rels2=[<Relationship element_id='9482136' nodes=(<Node element_id='200' labels=frozenset() properties={}>, <Node element_id='0' labels=frozenset() properties={}>) type='IN' properties={}>, <Relationship element_id='9458134' nodes=(<Node element_id='38210' labels=frozenset() properties={}>, <Node element_id='200' label

In [44]:
# 2.2. Find duplicate nodes if any (STYLES & USERS)
query = """
    // Find the total number of duplicate STYLES nodes
    MATCH (s:STYLES)
    WITH s.name AS styleName, count(s) AS count
    WHERE count > 1
    RETURN count(styleName) AS totalDuplicatesStyles
"""
result = execute_read(driver, query)
pprint(result)

query = """
    // Find the total number of duplicate USER nodes
    MATCH (u:USER)
    WITH u.name AS userName, count(u) AS count
    WHERE count > 1
    RETURN count(userName) AS totalDuplicatesUsers
"""
result = execute_read(driver, query)
pprint(result)

[<Record totalDuplicatesStyles=0>]
[<Record totalDuplicatesUsers=0>]


> As we already saw in the ***Statistical Analysis of the Database***, we don't have any duplicate **`STYLE`** and **`USER`** nodes.

In [45]:
# 2.2. Find duplicate nodes if any (REVIEWS)
query = """
    // Find duplicate REVIEWS nodes
    MATCH (r:REVIEWS)
    WITH r.id AS reviewId, count(r) AS count
    WHERE count > 1
    RETURN reviewId, count
    LIMIT 10
"""
result = execute_read(driver, query)
pprint(result)

query = """
    // Find the total number of duplicate REVIEWS nodes
    MATCH (r:REVIEWS)
    WITH r.id AS reviewId, count(r) AS count
    WHERE count > 1
    RETURN count(reviewId) AS totalDuplicates
"""
result = execute_read(driver, query)
pprint(result)

[<Record reviewId=6 count=2>,
 <Record reviewId=12 count=2>,
 <Record reviewId=28 count=2>,
 <Record reviewId=32 count=2>,
 <Record reviewId=45 count=2>,
 <Record reviewId=60 count=2>,
 <Record reviewId=86 count=2>,
 <Record reviewId=90 count=2>,
 <Record reviewId=92 count=2>,
 <Record reviewId=107 count=2>]
[<Record totalDuplicates=3111>]


> In total we have $3\; 111$ duplicate **`REVIEWS`** ($\frac{3\;111}{2\;549\;252} \approx 0.1\%$)

In [46]:
# Verify if the duplicate nodes have same relationships (REVIEWS)
query = """
    // Verify if the duplicate nodes have the same relationships
    MATCH (r:REVIEWS)
    WITH r.id AS reviewId, count(r) AS count, collect(r) AS nodes
    WHERE count > 1
    WITH reviewId, nodes
    UNWIND nodes AS node1
    UNWIND nodes AS node2
    WITH reviewId, node1, node2
    WHERE id(node1) <> id(node2)
    // Collect relationships for each node
    OPTIONAL MATCH (node1)-[r1]-()
    OPTIONAL MATCH (node2)-[r2]-()
    WITH reviewId, node1, node2, collect(r1) AS rels1, collect(r2) AS rels2
    // Compare relationships
    WHERE rels1 <> rels2
    RETURN reviewId, id(node1) AS id1, id(node2) AS id2, rels1, rels2
    LIMIT 2
"""
result = execute_read(driver, query)
pprint(result)

query = """
    // Verify if the duplicate nodes have the same relationships
    MATCH (r:REVIEWS)
    WITH r.id AS reviewId, count(r) AS count, collect(r) AS nodes
    WHERE count > 1
    WITH reviewId, nodes
    UNWIND nodes AS node1
    UNWIND nodes AS node2
    WITH reviewId, node1, node2
    WHERE id(node1) <> id(node2)
    // Collect relationships for each node
    OPTIONAL MATCH (node1)-[r1]-()
    OPTIONAL MATCH (node2)-[r2]-()
    WITH reviewId, node1, node2, collect(r1) AS rels1, collect(r2) AS rels2
    // Compare relationships
    WHERE rels1 <> rels2
    // Count how many cases
    RETURN count(reviewId) AS totalCases
"""
result = execute_read(driver, query)
pprint(result)

[<Record reviewId=6 id1=421091 id2=10080352 rels1=[<Relationship element_id='9853094' nodes=(<Node element_id='421091' labels=frozenset() properties={}>, <Node element_id='9494332' labels=frozenset() properties={}>) type='POSTED' properties={}>, <Relationship element_id='14' nodes=(<Node element_id='384103' labels=frozenset() properties={}>, <Node element_id='421091' labels=frozenset() properties={}>) type='REVIEWED' properties={}>] rels2=[]>,
 <Record reviewId=6 id1=10080352 id2=421091 rels1=[] rels2=[<Relationship element_id='9853094' nodes=(<Node element_id='421091' labels=frozenset() properties={}>, <Node element_id='9494332' labels=frozenset() properties={}>) type='POSTED' properties={}>, <Relationship element_id='14' nodes=(<Node element_id='384103' labels=frozenset() properties={}>, <Node element_id='421091' labels=frozenset() properties={}>) type='REVIEWED' properties={}>]>]
[<Record totalCases=6222>]


##### **Summary Table of Duplicate Nodes**

<center>

| Nodes           | Total    | Duplicates | Percentage |
|:---------------:|:--------:|:----------:|:----------:|
| **`BEERS`**     | 417,746  | 58,873     | 14.1%      |
| **`BREWERIES`** | 100,694  | 50,347     | 50.0%      |
| **`COUNTRIES`** | 400      | 200        | 50.0%      |
| **`CITIES`**    | 23,330   | 11,665     | 50.0%      |
| **`STYLE`**     | 113      | 0          | 0.0%       |
| **`USER`**      | 123,935  | 0          | 0.0%       |
| **`REVIEWS`**   | 2,549,252| 3,111      | 0.1%       |

</center>

---

#### **2.2.1. 🗑️ Remove Duplicate Nodes**

In [47]:
# Test the code 
query = """MATCH (b:BEERS) WITH b.id AS beerId, count(b) AS count WHERE count > 1 RETURN beerId, count LIMIT 1"""
result = execute_read(driver, query)
pprint(result)                           # beerId=214879

# Compare relationships of the duplicate node beerId=214879 as we did before
query = """
    MATCH (b:BEERS)
    WHERE b.id = 214879
    OPTIONAL MATCH (b)-[r1]-()
    RETURN b, collect(r1) AS rels
"""
result = execute_read(driver, query)
pprint(result)                           # Node element_id='9721475' dont have relationships

[<Record beerId=214879 count=2>]
[<Record b=<Node element_id='62214' labels=frozenset({'BEERS'}) properties={'notes': 'No notes at this time.', 'abv': 4.0, 'name': 'Scottish Right', 'retired': True, 'state': 'IN', 'id': 214879, 'brewery_id': 44306, 'availability': ' Year-round'}> rels=[<Relationship element_id='9494215' nodes=(<Node element_id='62214' labels=frozenset({'BEERS'}) properties={'notes': 'No notes at this time.', 'abv': 4.0, 'name': 'Scottish Right', 'retired': True, 'state': 'IN', 'id': 214879, 'brewery_id': 44306, 'availability': ' Year-round'}>, <Node element_id='9494215' labels=frozenset() properties={}>) type='HAS_STYLE' properties={}>, <Relationship element_id='9072917' nodes=(<Node element_id='13515' labels=frozenset() properties={}>, <Node element_id='62214' labels=frozenset({'BEERS'}) properties={'notes': 'No notes at this time.', 'abv': 4.0, 'name': 'Scottish Right', 'retired': True, 'state': 'IN', 'id': 214879, 'brewery_id': 44306, 'availability': ' Year-round'}>

In [48]:
query = """
    MATCH (b:BEERS)
    WITH b.id AS beerId, collect(b) AS nodes
    WHERE size(nodes) > 1
    WITH beerId, nodes[0] AS keepNode, nodes[1..] AS deleteNodes
    
    // Collect relationships of the node to keep
    OPTIONAL MATCH (keepNode)-[rKeep]->(target)
    WITH beerId, keepNode, deleteNodes, collect(rKeep) AS keepRels, collect(target) AS keepTargets
    OPTIONAL MATCH (source)-[rKeepIn]->(keepNode)
    WITH beerId, keepNode, deleteNodes, keepRels, keepTargets, collect(rKeepIn) AS keepRelsIn, collect(source) AS keepSources
    
    // Process each node to delete
    UNWIND deleteNodes AS delNode
    
    // Collect relationships of the node to delete
    OPTIONAL MATCH (delNode)-[rDel]->(delTarget)
    WITH beerId, keepNode, delNode, keepRels, keepTargets, keepRelsIn, keepSources,
         collect(rDel) AS delRels, collect(delTarget) AS delTargets
    OPTIONAL MATCH (delSource)-[rDelIn]->(delNode)
    WITH beerId, keepNode, delNode, keepRels, keepTargets, keepRelsIn, keepSources,
         delRels, delTargets, collect(rDelIn) AS delRelsIn, collect(delSource) AS delSources
    
    // Only proceed if delNode has no relationships and keepNode has some
    WHERE size(delRels) = 0 AND size(delRelsIn) = 0 AND (size(keepRels) > 0 OR size(keepRelsIn) > 0)
   
    // For debugging: return what was kept and deleted
    RETURN beerId, keepNode, delNode
    LIMIT 1                                          // Limit for testing
"""
result = execute_read(driver, query)

print("Nodes to keep and delete:")
for rec in result:
     print("Beer ID:", rec["beerId"])
     print("Keep Node:", rec["keepNode"])
     print("Delete Node:", rec["delNode"])

Nodes to keep and delete:
Beer ID: 214879
Keep Node: <Node element_id='62214' labels=frozenset({'BEERS'}) properties={'notes': 'No notes at this time.', 'abv': 4.0, 'name': 'Scottish Right', 'retired': True, 'state': 'IN', 'id': 214879, 'brewery_id': 44306, 'availability': ' Year-round'}>
Delete Node: <Node element_id='9721475' labels=frozenset({'BEERS'}) properties={'notes': 'No notes at this time.', 'abv': 4.0, 'name': 'Scottish Right', 'retired': True, 'state': 'IN', 'id': 214879, 'brewery_id': 44306, 'availability': ' Year-round'}>


In [49]:
# 2.2.1.1. (BEERS - Duplicate property: id) - Merge duplicates and delete nodes with no relationships 
query = """
    MATCH (b:BEERS)
    WITH b.id AS beerId, collect(b) AS nodes
    WHERE size(nodes) > 1
    WITH beerId, nodes[0] AS keepNode, nodes[1..] AS deleteNodes
    
    // Collect relationships of the node to keep
    OPTIONAL MATCH (keepNode)-[rKeep]->(target)
    WITH beerId, keepNode, deleteNodes, collect(rKeep) AS keepRels, collect(target) AS keepTargets
    OPTIONAL MATCH (source)-[rKeepIn]->(keepNode)
    WITH beerId, keepNode, deleteNodes, keepRels, keepTargets, collect(rKeepIn) AS keepRelsIn, collect(source) AS keepSources
    
    // Process each node to delete
    UNWIND deleteNodes AS delNode
    
    // Collect relationships of the node to delete
    OPTIONAL MATCH (delNode)-[rDel]->(delTarget)
    WITH beerId, keepNode, delNode, keepRels, keepTargets, keepRelsIn, keepSources,
         collect(rDel) AS delRels, collect(delTarget) AS delTargets
    OPTIONAL MATCH (delSource)-[rDelIn]->(delNode)
    WITH beerId, keepNode, delNode, keepRels, keepTargets, keepRelsIn, keepSources,
         delRels, delTargets, collect(rDelIn) AS delRelsIn, collect(delSource) AS delSources
    
    // Only proceed if delNode has no relationships and keepNode has some
    WHERE size(delRels) = 0 AND size(delRelsIn) = 0 AND (size(keepRels) > 0 OR size(keepRelsIn) > 0)
    
    // Delete the node with no relationships
    DELETE delNode
    
    // For debugging: return what was kept and deleted
    // RETURN beerId, keepNode, delNode
    // LIMIT 1                                          // Limit for testing
    
    // Return the number of nodes deleted
    RETURN count(delNode) AS nodesDeleted   
"""
result = execute_write(driver, query)
pprint(result)

[<Record nodesDeleted=58873>]


In [50]:
# Check if all BEERS nodes have relationships
query = """
    MATCH (b:BEERS)
    WITH count(b) AS totalNodes
    MATCH (b:BEERS)
    WHERE EXISTS { (b)-[]-() }
    RETURN count(b) AS totalNodesWithRelationships, totalNodes
"""
result = execute_read(driver, query)
pprint(result)

[<Record totalNodesWithRelationships=358873 totalNodes=358873>]


In [51]:
# Verify the changes
query = """
    MATCH (b:BEERS)
    WITH b.id AS beerId, count(b) AS count
    WHERE count > 1
    RETURN count(beerId) AS totalDuplicates
"""
result = execute_read(driver, query)
pprint(result)

[<Record totalDuplicates=0>]


In [52]:
# 2.2.1.2. (BREWERIES - Duplicate property: id) - Merge duplicates and delete nodes with no relationships
query = """
    MATCH (b:BREWERIES)
    WITH b.id AS breweryId, collect(b) AS nodes
    WHERE size(nodes) > 1
    WITH breweryId, nodes[0] AS keepNode, nodes[1..] AS deleteNodes
    // Collect relationships of the node to keep
    OPTIONAL MATCH (keepNode)-[rKeep]->(target)
    WITH breweryId, keepNode, deleteNodes, collect(rKeep) AS keepRels, collect(target) AS keepTargets
    OPTIONAL MATCH (source)-[rKeepIn]->(keepNode)
    WITH breweryId, keepNode, deleteNodes, keepRels, keepTargets, collect(rKeepIn) AS keepRelsIn, collect(source) AS keepSources
    // Process each node to delete
    UNWIND deleteNodes AS delNode
    // Collect relationships of the node to delete
    OPTIONAL MATCH (delNode)-[rDel]->(delTarget)
    WITH breweryId, keepNode, delNode, keepRels, keepTargets, keepRelsIn, keepSources,
         collect(rDel) AS delRels, collect(delTarget) AS delTargets
    OPTIONAL MATCH (delSource)-[rDelIn]->(delNode)
    WITH breweryId, keepNode, delNode, keepRels, keepTargets, keepRelsIn, keepSources,
         delRels, delTargets, collect(rDelIn) AS delRelsIn, collect(delSource) AS delSources
    // Only proceed if delNode has no relationships and keepNode has some
    WHERE size(delRels) = 0 AND size(delRelsIn) = 0 AND (size(keepRels) > 0 OR size(keepRelsIn) > 0)
    // Delete the node with no relationships
    DELETE delNode
    // Return the number of nodes deleted
    RETURN count(delNode) AS nodesDeleted
"""
result = execute_write(driver, query)
print("BREWERIES duplicates removed (no relationships):")
pprint(result)

BREWERIES duplicates removed (no relationships):
[<Record nodesDeleted=50347>]


In [53]:
# Check if all BREWERIES nodes have relationships
query = """
    MATCH (b:BREWERIES)
    WITH count(b) AS totalNodes
    MATCH (b:BREWERIES)
    WHERE EXISTS { (b)-[]-() }
    RETURN count(b) AS totalNodesWithRelationships, totalNodes
"""
result = execute_read(driver, query)
pprint(result)

[<Record totalNodesWithRelationships=50347 totalNodes=50347>]


In [54]:
# Verify the changes
query = """
    MATCH (b:BREWERIES)
    WITH b.id AS breweryId, count(b) AS count
    WHERE count > 1
    RETURN count(breweryId) AS totalDuplicates
"""
result = execute_read(driver, query)
pprint(result)

[<Record totalDuplicates=0>]


In [55]:
# 2.2.1.3. (CITIES - Duplicate property: name) - Merge duplicates and delete nodes with no relationships
query = """
    MATCH (c:CITIES)
    WITH c.name AS cityName, collect(c) AS nodes
    WHERE size(nodes) > 1
    WITH cityName, nodes[0] AS keepNode, nodes[1..] AS deleteNodes
    // Collect relationships of the node to keep
    OPTIONAL MATCH (keepNode)-[rKeep]->(target)
    WITH cityName, keepNode, deleteNodes, collect(rKeep) AS keepRels, collect(target) AS keepTargets
    OPTIONAL MATCH (source)-[rKeepIn]->(keepNode)
    WITH cityName, keepNode, deleteNodes, keepRels, keepTargets, collect(rKeepIn) AS keepRelsIn, collect(source) AS keepSources
    // Process each node to delete
    UNWIND deleteNodes AS delNode
    // Collect relationships of the node to delete
    OPTIONAL MATCH (delNode)-[rDel]->(delTarget)
    WITH cityName, keepNode, delNode, keepRels, keepTargets, keepRelsIn, keepSources,
         collect(rDel) AS delRels, collect(delTarget) AS delTargets
    OPTIONAL MATCH (delSource)-[rDelIn]->(delNode)
    WITH cityName, keepNode, delNode, keepRels, keepTargets, keepRelsIn, keepSources,
         delRels, delTargets, collect(rDelIn) AS delRelsIn, collect(delSource) AS delSources
    // Only proceed if delNode has no relationships and keepNode has some
    WHERE size(delRels) = 0 AND size(delRelsIn) = 0 AND (size(keepRels) > 0 OR size(keepRelsIn) > 0)
    // Delete the node with no relationships
    DELETE delNode
    // Return the number of nodes deleted
    RETURN count(delNode) AS nodesDeleted
"""
result = execute_write(driver, query)
print("CITIES duplicates removed (no relationships):")
pprint(result)

CITIES duplicates removed (no relationships):
[<Record nodesDeleted=11665>]


In [56]:
# Check if all CITIES nodes have relationships
query = """
    MATCH (c:CITIES)
    WITH count(c) AS totalNodes
    MATCH (c:CITIES)
    WHERE EXISTS { (c)-[]-() }
    RETURN count(c) AS totalNodesWithRelationships, totalNodes
"""
result = execute_read(driver, query)
pprint(result)

# Verify the changes
query = """
    MATCH (c:CITIES)
    WITH c.name AS cityName, count(c) AS count
    WHERE count > 1
    RETURN count(cityName) AS totalDuplicates
"""
result = execute_read(driver, query)
pprint(result)

[<Record totalNodesWithRelationships=11665 totalNodes=11665>]
[<Record totalDuplicates=0>]


In [57]:
# 2.2.1.4. (COUNTRIES - Duplicate property: name) - Merge duplicates and delete nodes with no relationships
query = """
    MATCH (c:COUNTRIES)
    WITH c.name AS countryName, collect(c) AS nodes
    WHERE size(nodes) > 1
    WITH countryName, nodes[0] AS keepNode, nodes[1..] AS deleteNodes
    // Collect relationships of the node to keep
    OPTIONAL MATCH (keepNode)-[rKeep]->(target)
    WITH countryName, keepNode, deleteNodes, collect(rKeep) AS keepRels, collect(target) AS keepTargets
    OPTIONAL MATCH (source)-[rKeepIn]->(keepNode)
    WITH countryName, keepNode, deleteNodes, keepRels, keepTargets, collect(rKeepIn) AS keepRelsIn, collect(source) AS keepSources
    // Process each node to delete
    UNWIND deleteNodes AS delNode
    // Collect relationships of the node to delete
    OPTIONAL MATCH (delNode)-[rDel]->(delTarget)
    WITH countryName, keepNode, delNode, keepRels, keepTargets, keepRelsIn, keepSources,
         collect(rDel) AS delRels, collect(delTarget) AS delTargets
    OPTIONAL MATCH (delSource)-[rDelIn]->(delNode)
    WITH countryName, keepNode, delNode, keepRels, keepTargets, keepRelsIn, keepSources,
         delRels, delTargets, collect(rDelIn) AS delRelsIn, collect(delSource) AS delSources
    // Only proceed if delNode has no relationships and keepNode has some
    WHERE size(delRels) = 0 AND size(delRelsIn) = 0 AND (size(keepRels) > 0 OR size(keepRelsIn) > 0)
    // Delete the node with no relationships
    DELETE delNode
    // Return the number of nodes deleted
    RETURN count(delNode) AS nodesDeleted
"""
result = execute_write(driver, query)
print("COUNTRIES duplicates removed (no relationships):")
pprint(result)

COUNTRIES duplicates removed (no relationships):
[<Record nodesDeleted=200>]


In [58]:
# Check if all COUNTRIES nodes have relationships
query = """
    MATCH (c:COUNTRIES)
    WITH count(c) AS totalNodes
    MATCH (c:COUNTRIES)
    WHERE EXISTS { (c)-[]-() }
    RETURN count(c) AS totalNodesWithRelationships, totalNodes
"""
result = execute_read(driver, query)
pprint(result)

# Verify the changes
query = """
    MATCH (c:COUNTRIES)
    WITH c.name AS countryName, count(c) AS count
    WHERE count > 1
    RETURN count(countryName) AS totalDuplicates
"""
result = execute_read(driver, query)
pprint(result)

[<Record totalNodesWithRelationships=200 totalNodes=200>]
[<Record totalDuplicates=0>]


In [59]:
# 2.2.1.5. (REVIEWS - Duplicate property: id) - Merge duplicates and delete nodes with no relationships
query = """
    MATCH (r:REVIEWS)
    WITH r.id AS reviewId, collect(r) AS nodes
    WHERE size(nodes) > 1
    WITH reviewId, nodes[0] AS keepNode, nodes[1..] AS deleteNodes
    // Collect relationships of the node to keep
    OPTIONAL MATCH (keepNode)-[rKeep]->(target)
    WITH reviewId, keepNode, deleteNodes, collect(rKeep) AS keepRels, collect(target) AS keepTargets
    OPTIONAL MATCH (source)-[rKeepIn]->(keepNode)
    WITH reviewId, keepNode, deleteNodes, keepRels, keepTargets, collect(rKeepIn) AS keepRelsIn, collect(source) AS keepSources
    // Process each node to delete
    UNWIND deleteNodes AS delNode
    // Collect relationships of the node to delete
    OPTIONAL MATCH (delNode)-[rDel]->(delTarget)
    WITH reviewId, keepNode, delNode, keepRels, keepTargets, keepRelsIn, keepSources,
         collect(rDel) AS delRels, collect(delTarget) AS delTargets
    OPTIONAL MATCH (delSource)-[rDelIn]->(delNode)
    WITH reviewId, keepNode, delNode, keepRels, keepTargets, keepRelsIn, keepSources,
         delRels, delTargets, collect(rDelIn) AS delRelsIn, collect(delSource) AS delSources
    // Only proceed if delNode has no relationships and keepNode has some
    WHERE size(delRels) = 0 AND size(delRelsIn) = 0 AND (size(keepRels) > 0 OR size(keepRelsIn) > 0)
    // Delete the node with no relationships
    DELETE delNode
    // Return the number of nodes deleted
    RETURN count(delNode) AS nodesDeleted
"""
result = execute_write(driver, query)
print("REVIEWS duplicates removed (no relationships):")
pprint(result)

REVIEWS duplicates removed (no relationships):
[<Record nodesDeleted=3111>]


In [60]:
# Check if all REVIEWS nodes have relationships
query = """
    MATCH (r:REVIEWS)
    WITH count(r) AS totalNodes
    MATCH (r:REVIEWS)
    WHERE EXISTS { (r)-[]-() }
    RETURN count(r) AS totalNodesWithRelationships, totalNodes
"""
result = execute_read(driver, query)
pprint(result)

# Verify the changes
query = """
    MATCH (r:REVIEWS)
    WITH r.id AS reviewId, count(r) AS count
    WHERE count > 1
    RETURN count(reviewId) AS totalDuplicates
"""
result = execute_read(driver, query)
pprint(result)

[<Record totalNodesWithRelationships=2538044 totalNodes=2546160>]
[<Record totalDuplicates=0>]


In [61]:
# [<Record totalNodesWithRelationships=2538044 totalNodes=2546160>] - There are REVIEW nodes without relationships
# Nodes without relationships: 2,546,160 - 2,538,044 = 8,116.

query = """
    // Count and sample REVIEWS nodes without relationships
    MATCH (r:REVIEWS)
    WHERE NOT EXISTS { (r)-[]-() }
    WITH count(r) AS nodesWithoutRelationships, collect(r) AS nodes
    RETURN nodesWithoutRelationships, nodes[0..5] AS sampleNodes        // Sample 5 nodes for inspection | Source: https://stackoverflow.com/questions/19075817/retrieving-subset-of-array-list-on-neo4j-node-property
"""
result = execute_read(driver, query)
pprint(result)

[<Record nodesWithoutRelationships=8116 sampleNodes=[<Node element_id='921375' labels=frozenset() properties={}>, <Node element_id='921921' labels=frozenset() properties={}>, <Node element_id='922467' labels=frozenset() properties={}>, <Node element_id='923013' labels=frozenset() properties={}>, <Node element_id='923559' labels=frozenset() properties={}>]>]


In [62]:
# Remove the nodes from the previous query
query = """
    // Remove REVIEWS nodes without relationships
    MATCH (r:REVIEWS)
    WHERE NOT EXISTS { (r)-[]-() }
    DETACH DELETE r
"""
result = execute_write(driver, query)
pprint(result)

[]


In [63]:
# Check if nodes of REVIEWS without relationships were removed
query = """
    MATCH (r:REVIEWS)
    WHERE NOT EXISTS { (r)-[]-() }
    RETURN count(r) AS totalNodesWithoutRelationships
"""
result = execute_read(driver, query)
pprint(result)

[<Record totalNodesWithoutRelationships=19>]


### **2.3 Detect and Remove Errors**

In [64]:
# 2.3.1. Nodes without labels
query = """
    MATCH (n)
    WHERE isEmpty(labels(n))
    WITH count(n) AS totalNodesWithoutLabels, collect(n) AS nodes
    RETURN totalNodesWithoutLabels, nodes[0..5] AS sampleNodes
"""
result = execute_read(driver, query)
print("Nodes without labels - Count and 5 examples:")
pprint(result)

Nodes without labels - Count and 5 examples:
[<Record totalNodesWithoutLabels=0 sampleNodes=[]>]


In [65]:
# Re-do the query 1.11 to see Min_RelationshipCount and Max_RelationshipCount
query = """
    // What kind of nodes exist
    // Sample some nodes, reporting on property and relationship counts per node.
    MATCH (n) WHERE rand() <= 0.1
    RETURN
    DISTINCT labels(n),
    count(*) AS SampleSize,
    avg(size(keys(n))) as Avg_PropertyCount,
    min(size(keys(n))) as Min_PropertyCount,
    max(size(keys(n))) as Max_PropertyCount,
    avg(size( (n)-[]-() ) ) as Avg_RelationshipCount,
    min(size( (n)-[]-() ) ) as Min_RelationshipCount,
    max(size( (n)-[]-() ) ) as Max_RelationshipCount
"""
result = execute_read(driver, query)
data = [dict(record) for record in result]
df = pd.DataFrame(data)
df.set_index("labels(n)", inplace=True)
df.index.name = "Node Label"
df

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: } {title: This feature is deprecated and will be removed in future versions.} {description: A pattern expression should only be used in order to test the existence of a pattern. It should therefore only be used in contexts that evaluate to a boolean, e.g. inside the function exists() or in a WHERE-clause. All other uses are deprecated and should be replaced by a pattern comprehension.} {position: line: 11, column: 15, offset: 361} for query: '\n    // What kind of nodes exist\n    // Sample some nodes, reporting on property and relationship counts per node.\n    MATCH (n) WHERE rand() <= 0.1\n    RETURN\n    DISTINCT labels(n),\n    count(*) AS SampleSize,\n    avg(size(keys(n))) as Avg_PropertyCount,\n    min(size(keys(n))) as Min_PropertyCount,\n    max(size(keys(n))) as Max_PropertyCount,\n    avg(size( (n)-[]-() ) ) as Avg_RelationshipCount,\n   

,SampleSize,Avg_PropertyCount,Min_PropertyCount,Max_PropertyCount,Avg_RelationshipCount,Min_RelationshipCount,Max_RelationshipCount
Node Label,,,,,,,
[COUNTRIES],15,1.000000,1,1,19.333333,1,158
[CITIES],1178,1.000000,1,1,5.067063,2,182
[BREWERIES],5014,5.000000,5,5,8.777623,1,923
[BEERS],35871,7.892058,7,8,9.126648,2,4396
[REVIEWS],253773,7.910475,5,10,1.999976,1,2
[STYLE],7,1.000000,1,1,2337.142857,1333,3801
[USER],12438,1.000000,1,1,20.528783,0,2344


##### **Comments:**

- **`Min_PropertyCount`**:
  - After removing duplicates, the `Min_PropertyCount` increased for some node labels, indicating that some nodes with fewer properties were duplicates and have been removed.
  - However, there are still `REVIEWS` and `USER` nodes with no relationships (`Min_RelationshipCount = 0`). This suggests that while duplicates were addressed, some nodes remain unconnected in the graph.

- **`Max_RelationshipCount` for `REVIEWS`**:
  - The `Max_RelationshipCount` for `REVIEWS` is 2, which is not expected since the primary relationship for `REVIEWS` is `REVIEWS - POSTED -> USER`.

In [66]:
# 2.3 - REVIEW nodes without relationships
query = """
    // Count and sample REVIEW nodes without relationships
    MATCH (r:REVIEWS)
    WHERE NOT EXISTS { (r)-[]-() }
    WITH count(r) AS totalReviewsWithoutRelationships, collect(r) AS reviewNodes
    RETURN totalReviewsWithoutRelationships, reviewNodes[0..2] AS sampleReviewNodes
"""
result = execute_read(driver, query)
print("REVIEW nodes without relationships - Count and 2 examples:")
pprint(result)

REVIEW nodes without relationships - Count and 2 examples:
[<Record totalReviewsWithoutRelationships=19 sampleReviewNodes=[<Node element_id='921375' labels=frozenset() properties={}>, <Node element_id='921921' labels=frozenset() properties={}>]>]


In [67]:
# Query to remove REVIEW nodes without relationships and without properties
query = """
    MATCH (r:REVIEWS)
    WHERE NOT EXISTS((r)-[]-()) AND size(keys(r)) = 0
    DELETE r
    RETURN count(r) AS totalNodesDeleted
"""

# Execute the query
try:
    result = execute_write(driver, query)
    print(f"Total REVIEW nodes deleted (without relationships and properties): {result[0]['totalNodesDeleted']}")
except Exception as e:
    print(f"Error removing REVIEW nodes: {e}")

Total REVIEW nodes deleted (without relationships and properties): 19


In [68]:
# Verify if nodes of REVIEWS without relationships were removed
query = """
    // Check if REVIEW nodes without relationships were removed
    MATCH (r:REVIEWS)
    WHERE NOT EXISTS { (r)-[]-() }
    RETURN count(r) AS totalNodesWithoutRelationships
"""
result = execute_read(driver, query)
pprint(result)

[<Record totalNodesWithoutRelationships=19>]


<div class="alert alert-block alert-warning">

## **PERGUNTAR PROFª**

In [69]:
# 2.3. USER nodes without relationships 
query = """
    // Count and sample USER nodes without relationships
    MATCH (u:USER)
    WHERE NOT EXISTS { (u)-[]-() }
    WITH count(u) AS totalUsersWithoutRelationships, collect(u) AS userNodes
    RETURN totalUsersWithoutRelationships, userNodes[0..5] AS sampleUserNodes
"""
result = execute_read(driver, query)
print("USER nodes without relationships - Count and 5 examples:")
pprint(result)

USER nodes without relationships - Count and 5 examples:
[<Record totalUsersWithoutRelationships=17290 sampleUserNodes=[<Node element_id='9494578' labels=frozenset({'USER'}) properties={'name': 'Rick_Ereth'}>, <Node element_id='9495077' labels=frozenset({'USER'}) properties={'name': 'matttyt'}>, <Node element_id='9495143' labels=frozenset({'USER'}) properties={'name': 'ChaBrah'}>, <Node element_id='9495174' labels=frozenset({'USER'}) properties={'name': 'bbc0202'}>, <Node element_id='9495303' labels=frozenset({'USER'}) properties={'name': 'Kbenoit16'}>]>]


- Since we don't know the exact origin of this data and where/how it was obtained, it might make sense for there to be **`USERS`** with no reviews (e.g., users who have created an account on the data retrieval page, but have never done a REVIEW). So, we will not remove these nodes.


---

In [70]:
# 2.3. BEERS.abv greater than 67.5
query = """
    MATCH (b:BEERS)
    WHERE b.abv > 67.5 OR b.abv < 0
    RETURN b.name, b.abv
    ORDER BY b.abv DESC
"""
result = execute_read(driver, query)
pprint(result)

[<Record b.name="Earache: World's Shortest Album" b.abv=100.0>,
 <Record b.name='Radiohead - OK Computer' b.abv=100.0>,
 <Record b.name='water' b.abv=100.0>,
 <Record b.name='Dark Reckoning' b.abv=80.0>]


Checking the **`BEERS`** node, we found that there are some beers with **`abv`** greater than **$67.5$**. After observing that cases, we found that the names of the beers are **`Earache: World's Shortest Album`** and **`Dark Reckoning`**, **`Radiohead - OK Computer`** and **`water`** and we researched about them and found that they are not beers, so we will remove them.

In [71]:
# 2.3. Remove nodes with BEERS.abv greater than 67.5
query = """
    MATCH (b:BEERS)
    WHERE b.abv > 67.5 OR b.abv < 0
    DETACH DELETE b
"""
result = execute_write(driver, query)
pprint(result)

[]


In [72]:
# Verify the changes 
query = """
    // Count the number of BEERS nodes with abv > 67.5 and return the total and a sample
    MATCH (b:BEERS)
    WHERE b.abv > 67.5
    WITH count(b) AS totalBeers, collect(b) AS beers
    RETURN totalBeers, beers[0..5] AS sampleBeers
"""
result = execute_read(driver, query)
pprint(result)

[<Record totalBeers=0 sampleBeers=[]>]


### **2.4. 🧹 Remove Missing Values**

In [ ]:
# Source: https://neo4j.com/docs/cypher-manual/current/clauses/remove/

# 2.4 - Re-run the query 1.12 (Before Missing Values Handling)

print("\n\nSummary statistics of each property for each node label in the database (After Cleaning):")

# Initialize a list to store statistics
stats_list = []

# Iterate over each property and calculate statistics
for record in tqdm(properties_result):
    nodeType = record['nodeType'].replace('`', '').replace(':', '')  # Remove backticks and ':'
    propertyName = record['propertyName']
    
    # Construct query for each property
    if propertyName.lower() in ['name', 'notes', 'state', 'availability', 'types', 'text']:
        # String properties
        stats_query = f"""
            MATCH (n:{nodeType})
            RETURN
                '{nodeType}' AS nodeType,
                '{propertyName}' AS propertyName,
                COUNT(n.{propertyName}) AS count,
                COUNT(DISTINCT n.{propertyName}) AS distinctCount,
                '-' AS mean,
                '-' AS min,
                '-' AS max,
                COUNT(CASE WHEN n.{propertyName} IS NULL OR n.{propertyName} = '' OR toLower(trim(n.{propertyName})) IN ['nan', 'null', 'none', '', ' '] THEN 1 ELSE NULL END) AS missingCount
        """
    elif propertyName.lower() in ['retired', 'date']:
        # Boolean & Date properties
        stats_query = f"""
            MATCH (n:{nodeType})
            RETURN
                '{nodeType}' AS nodeType,
                '{propertyName}' AS propertyName,
                COUNT(n.{propertyName}) AS count,
                COUNT(DISTINCT n.{propertyName}) AS distinctCount,
                '-' AS mean,
                '-' AS min,
                '-' AS max,
                COUNT(CASE WHEN n.{propertyName} IS NULL THEN 1 ELSE NULL END) AS missingCount
        """
    else:
        # Numeric properties
        stats_query = f"""
            MATCH (n:{nodeType})
            WITH n, n.{propertyName} AS value
            RETURN
                '{nodeType}' AS nodeType,
                '{propertyName}' AS propertyName,
                COUNT(value) AS count,
                COUNT(DISTINCT value) AS distinctCount,
                AVG(value) AS mean,
                MIN(value) AS min,
                MAX(value) AS max,
                COUNT(CASE WHEN value IS NULL THEN 1 ELSE NULL END) AS missingCount
        """
    
    try:
        stats_result = execute_read(driver, stats_query)
        if stats_result:
            stats_list.append(stats_result[0])
    except Exception as e:
        print(f"Error processing {nodeType}.{propertyName}: {e}")

# Convert to DataFrame
df = pd.DataFrame(stats_list, columns=['Node', 'Property', 'Count', 'Distinct Count', 
                                       'Mean', 'Min', 'Max', 'Missing Count'])                          # Convert the list to a DataFrame
df['Missing Count (%)'] = (df['Missing Count'] / df['Count']) * 100                                     # Calculate the percentage of missing values
df.set_index(['Node', 'Property'], inplace=True)                                                        # Set the index to Node and Property (Hierarchical Indexing)
df.fillna('-', inplace=True)                                                                            # Fill NaN values with 0
df = df.map(
    lambda x: '{:,.0f}'.format(x).replace(',', ' ') if isinstance(x, float) and not np.isnan(x) and not np.isinf(x) and x == int(x) else
              '{:,.2f}'.format(x).replace(',', ' ') if isinstance(x, float) and not np.isnan(x) and not np.isinf(x) else
              x
)                                                                                                                     # Format the DataFrame for better visualization
df 



Summary statistics of each property for each node label in the database (After Cleaning):


100%|██████████| 27/27 [06:36<00:00, 14.68s/it]


Count  Distinct Count          Mean   Min      Max  \
Node      Property                                                             
BEERS     abv            320072             937          6.53  0.01    67.50   
          availability   358869              20             -     -        -   
          brewery_id     358869           16569     24 597.39     1    54144   
          id             358869          358869    189 239.20     3   374406   
          name           358869          298564             -     -        -   
          notes          358869           48313             -     -        -   
          retired        358869               2             -     -        -   
          state          358869              68             -     -        -   
BREWERIES id              50347           50347     27 870.51     1    54156   
          name            50347           45245             -     -        -   
          notes           50347            3271             -     -        -   
          state           50347              68             -     -        -   
          types           50347              30             -     -        -   
CITIES    name            11665           11665             -     -        -   
COUNTRIES name              200             200             -     -        -   
REVIEWS   beer_id       2538044          189395     77 311.79     3   373128   
          date          2538044            6379             -     -        -   
          feel          1477561              17          3.88     1        5   
          id            2538044         2538044  4 537 303.98     1  9073127   
          look          1477561              17          3.95     1        5   
          overall       1477561              17          3.92     1        5   
          score         2538044             401          3.89     1        5   
          smell         1477561              17          3.89     1        5   
          taste         1477561              17          3.92     1        5   
          text          2538044          812239             -     -        -   
STYLE     name              113             113             -     -        -   
USER      name           123935          123935             -     -        -   

                        Missing Count Missing Count (%)  
Node      Property                                       
BEERS     abv                   38797             12.12  
          availability              0                 0  
          brewery_id                0                 0  
          id                        0                 0  
          name                      0                 0  
          notes                    46              0.01  
          retired                   0                 0  
          state                 60726             16.92  
BREWERIES id                        0                 0  
          name                      0                 0  
          notes                    85              0.17  
          state                 11271             22.39  
          types                     0                 0  
CITIES    name                      1              0.01  
COUNTRIES name                      1              0.50  
REVIEWS   beer_id                  19              0.00  
          date                     19              0.00  
          feel                1060502             71.77  
          id                       19              0.00  
          look                1060502             71.77  
          overall             1060502             71.77  
          score                    19              0.00  
          smell               1060502             71.77  
          taste               1060502             71.77  
          text                     19              0.00  
STYLE     name                      1              0.88  
USER      name                      1              0.00

In [74]:
# Removing missing properties from nodes and logging changes

# List of nodes and properties with missing values
missing_values = {
    "BEERS": ["abv", "state", "notes"],
    "BREWERIES": ["notes", "state"],
    "CITIES": ["name"],
    "COUNTRIES": ["name"],
    "REVIEWS": ["beer_id", "date", "feel", "id", "look", "overall", "score", "smell", "taste", "text"],
    "STYLE": ["name"],
    "USER": ["name"]
}

# Dictionary to store how many nodes had properties removed
nodes_updated_count = {}

# Iterate over each node type and property to remove the missing property values
for node_type, properties in missing_values.items():
    for property in properties:
        # Handle STRING properties
        if property in ['name', 'notes', 'state', 'availability', 'types', 'text']:  
            condition = f"n.{property} IS NULL OR n.{property} = '' OR toLower(trim(COALESCE(n.{property}, ''))) IN ['nan', 'null', 'none', '', 'n/a', 'na']"
        # Handle NUMERIC properties (Double, Long)
        elif property in ['abv', 'beer_id', 'score', 'feel', 'look', 'overall', 'smell', 'taste']:
            condition = f"n.{property} IS NULL"
        # Handle DATE properties
        elif property in ['date']:
            condition = f"n.{property} IS NULL"
        else:
            continue  # Skip unknown property types

        # Query to count affected nodes
        count_query = f"""
            MATCH (n:{node_type})
            WHERE {condition}
            RETURN count(n) AS nodesUpdated
        """

        # Execute count query
        try:
            count_result = execute_read(driver, count_query)
            if count_result and count_result[0]['nodesUpdated'] > 0:
                affected_nodes = count_result[0]['nodesUpdated']

                # Query to remove the property
                # Source: https://neo4j.com/docs/cypher-manual/current/clauses/remove/
                remove_query = f"""
                    MATCH (n:{node_type})
                    WHERE {condition}
                    REMOVE n.{property}
                """
                
                # Execute remove query
                execute_write(driver, remove_query)

                # Store the count of affected nodes
                if node_type not in nodes_updated_count:
                    nodes_updated_count[node_type] = 0
                nodes_updated_count[node_type] += affected_nodes

        except Exception as e:
            print(f"Error processing {node_type} nodes with missing {property}: {e}")

# Print summary of changes
print("\nSummary of nodes where properties were removed:")
for node, count in nodes_updated_count.items():
    print(f"Label: {node}, Nodes Updated: {count}")


Summary of nodes where properties were removed:
Label: BEERS, Nodes Updated: 99569
Label: BREWERIES, Nodes Updated: 11356
Label: CITIES, Nodes Updated: 1
Label: COUNTRIES, Nodes Updated: 1
Label: REVIEWS, Nodes Updated: 5302586
Label: STYLE, Nodes Updated: 1
Label: USER, Nodes Updated: 1


In [75]:
# Re-run the query to see the changes after removing missing values
print("\n\nSummary statistics of each property for each node label in the database (After Cleaning):")

# Initialize a list to store statistics
stats_list = []

# Iterate over each property and calculate statistics
for record in tqdm(properties_result):
    nodeType = record['nodeType'].replace('`', '').replace(':', '')  # Remove backticks and ':'
    propertyName = record['propertyName']
    
    # Construct query for each property
    if propertyName.lower() in ['name', 'notes', 'state', 'availability', 'types', 'text']:
        # String properties
        stats_query = f"""
            MATCH (n:{nodeType})
            RETURN
                '{nodeType}' AS nodeType,
                '{propertyName}' AS propertyName,
                COUNT(n.{propertyName}) AS count,
                COUNT(DISTINCT n.{propertyName}) AS distinctCount,
                '-' AS mean,
                '-' AS min,
                '-' AS max,
                COUNT(CASE WHEN n.{propertyName} IS NULL OR n.{propertyName} = '' OR toLower(trim(n.{propertyName})) IN ['nan', 'null', 'none', '', ' '] THEN 1 ELSE NULL END) AS missingCount
        """
    elif propertyName.lower() in ['retired', 'date']:
        # Boolean & Date properties
        stats_query = f"""
            MATCH (n:{nodeType})
            RETURN
                '{nodeType}' AS nodeType,
                '{propertyName}' AS propertyName,
                COUNT(n.{propertyName}) AS count,
                COUNT(DISTINCT n.{propertyName}) AS distinctCount,
                '-' AS mean,
                '-' AS min,
                '-' AS max,
                COUNT(CASE WHEN n.{propertyName} IS NULL THEN 1 ELSE NULL END) AS missingCount
        """
    else:
        # Numeric properties
        stats_query = f"""
            MATCH (n:{nodeType})
            WITH n, n.{propertyName} AS value
            RETURN
                '{nodeType}' AS nodeType,
                '{propertyName}' AS propertyName,
                COUNT(value) AS count,
                COUNT(DISTINCT value) AS distinctCount,
                AVG(value) AS mean,
                MIN(value) AS min,
                MAX(value) AS max,
                COUNT(CASE WHEN value IS NULL THEN 1 ELSE NULL END) AS missingCount
        """
    
    try:
        stats_result = execute_read(driver, stats_query)
        if stats_result:
            stats_list.append(stats_result[0])
    except Exception as e:
        print(f"Error processing {nodeType}.{propertyName}: {e}")

# Convert to DataFrame
df = pd.DataFrame(stats_list, columns=['Node', 'Property', 'Count', 'Distinct Count', 
                                       'Mean', 'Min', 'Max', 'Missing Count'])                          # Convert the list to a DataFrame
df['Missing Count (%)'] = (df['Missing Count'] / df['Count']) * 100                                     # Calculate the percentage of missing values
df.set_index(['Node', 'Property'], inplace=True)                                                        # Set the index to Node and Property (Hierarchical Indexing)
df.fillna('-', inplace=True)                                                                            # Fill NaN values with 0
df = df.map(
    lambda x: '{:,.0f}'.format(x).replace(',', ' ') if isinstance(x, float) and not np.isnan(x) and not np.isinf(x) and x == int(x) else
              '{:,.2f}'.format(x).replace(',', ' ') if isinstance(x, float) and not np.isnan(x) and not np.isinf(x) else
              x
)                                                                                                                     # Format the DataFrame for better visualization
df 



Summary statistics of each property for each node label in the database (After Cleaning):


100%|██████████| 27/27 [12:28<00:00, 27.71s/it] 


Count  Distinct Count          Mean   Min      Max  \
Node      Property                                                             
BEERS     abv            320072             937          6.53  0.01    67.50   
          availability   358869              20             -     -        -   
          brewery_id     358869           16569     24 597.39     1    54144   
          id             358869          358869    189 239.20     3   374406   
          name           358869          298564             -     -        -   
          notes          358823           48312             -     -        -   
          retired        358869               2             -     -        -   
          state          298143              67             -     -        -   
BREWERIES id              50347           50347     27 870.51     1    54156   
          name            50347           45245             -     -        -   
          notes           50262            3270             -     -        -   
          state           39076              67             -     -        -   
          types           50347              30             -     -        -   
CITIES    name            11664           11664             -     -        -   
COUNTRIES name              199             199             -     -        -   
REVIEWS   beer_id       2538044          189395     77 311.79     3   373128   
          date          2538044            6379             -     -        -   
          feel          1477561              17          3.88     1        5   
          id            2538044         2538044  4 537 303.98     1  9073127   
          look          1477561              17          3.95     1        5   
          overall       1477561              17          3.92     1        5   
          score         2538044             401          3.89     1        5   
          smell         1477561              17          3.89     1        5   
          taste         1477561              17          3.92     1        5   
          text          2538044          812239             -     -        -   
STYLE     name              112             112             -     -        -   
USER      name           123934          123934             -     -        -   

                        Missing Count Missing Count (%)  
Node      Property                                       
BEERS     abv                   38797             12.12  
          availability              0                 0  
          brewery_id                0                 0  
          id                        0                 0  
          name                      0                 0  
          notes                    46              0.01  
          retired                   0                 0  
          state                 60726             20.37  
BREWERIES id                        0                 0  
          name                      0                 0  
          notes                    85              0.17  
          state                 11271             28.84  
          types                     0                 0  
CITIES    name                      1              0.01  
COUNTRIES name                      1              0.50  
REVIEWS   beer_id                  19              0.00  
          date                     19              0.00  
          feel                1060502             71.77  
          id                       19              0.00  
          look                1060502             71.77  
          overall             1060502             71.77  
          score                    19              0.00  
          smell               1060502             71.77  
          taste               1060502             71.77  
          text                     19              0.00  
STYLE     name                      1              0.89  
USER      name                      1              0.00

In [108]:
# Check only one exame of the label BEERS to see if the property abv was removed
query = """
    MATCH (b:BEERS)
    WHERE b.abv IS NULL
    RETURN b
    LIMIT 1
"""
result = execute_read(driver, query)
pprint(result)

[<Record b=<Node element_id='62233' labels=frozenset({'BEERS'}) properties={'notes': 'No notes at this time.', 'name': 'Cerveza India', 'retired': True, 'id': 15645, 'brewery_id': 846, 'availability': ' Year-round'}>>]


### **2.5. 📊 Change Schema** 

In [76]:
# Source: https://stackoverflow.com/questions/13816712/how-do-i-rename-relationships-in-neo4j

# MATCH (n1)-[old:OLDNAMERELATIONSHIP]->(n2)
# CREATE (n1)-[new:NEWNAMERELATIONSHIP]->(n2)
# DELETE old
# RETURN count(old) AS numDeletedRels, count(new) AS numNewRels

In [77]:
# 2.5. Visualize the final schema
query = """
    CALL db.schema.visualization()
"""
result = execute_read(driver, query)
print("Final database schema visualized (node labels and relationships):\n")
print("Nodes:", [record["nodes"] for record in result][0])
print("Relationships:", [record["relationships"] for record in result][0])

Final database schema visualized (node labels and relationships):

Nodes: [<Node element_id='-12' labels=frozenset({'REVIEWS'}) properties={'name': 'REVIEWS', 'indexes': ['id'], 'constraints': []}>, <Node element_id='-11' labels=frozenset({'BEERS'}) properties={'name': 'BEERS', 'indexes': ['id'], 'constraints': []}>, <Node element_id='-8' labels=frozenset({'COUNTRIES'}) properties={'name': 'COUNTRIES', 'indexes': ['name'], 'constraints': []}>, <Node element_id='-10' labels=frozenset({'BREWERIES'}) properties={'name': 'BREWERIES', 'indexes': ['id'], 'constraints': []}>, <Node element_id='-13' labels=frozenset({'STYLE'}) properties={'name': 'STYLE', 'indexes': ['name'], 'constraints': []}>, <Node element_id='-9' labels=frozenset({'CITIES'}) properties={'name': 'CITIES', 'indexes': ['name'], 'constraints': []}>, <Node element_id='-14' labels=frozenset({'USER'}) properties={'name': 'USER', 'indexes': ['name'], 'constraints': []}>]
Relationships: [<Relationship element_id='-9' nodes=(<Node 

<div class="alert alert-block alert-warning">

# **PERGUNTAR PROFª | A prof quer a imagem no final? Não dá para tirar as ligações que não existem , mas estão representadas! (??) e é preciso trocar as ligações?**

<img src="./img/TheBeerProject_Schema.png">

<img src="./img/TheBeerProject_FinalSchema.png" width="700">

</div>

---

## **3. Analytics department**

In [78]:
# 3. Analytics department requires the following information for the biweekly reporting: [5 points]
#    3.1. How many reviews has the beer with the most reviews?
query = """
    MATCH (b:BEERS)-[:REVIEWED]->(r:REVIEWS)
    RETURN b.name AS BeerName, count(r) AS NumberOfReviews
    ORDER BY NumberOfReviews DESC
    LIMIT 1
"""
result = execute_read(driver, query)

# Print the result
print("The beer with the most reviews is:", result[0]["BeerName"], "with", result[0]["NumberOfReviews"], "reviews.")

The beer with the most reviews is: IPA with 8771 reviews.


In [ ]:
# 3.2. Which three users wrote the most reviews about beers?
#           REVIEWS - POSTED -> USER
#           BEERS - REVIEWED -> REVIEWS

query = """
    MATCH (r:BEERS)-[:REVIEWED]->(b:REVIEWS)-[:POSTED]->(u:USER)
    RETURN u.name AS UserName, count(b) AS NumberOfReviews
    ORDER BY NumberOfReviews DESC
    LIMIT 3
"""
result = execute_read(driver, query)
# Print the result
print("The three users who wrote the most reviews are:", 
        "\n - \033[1m1st\033[0m", result[0]["UserName"], "with", result[0]["NumberOfReviews"], "reviews.",
        "\n - \033[1m2nd\033[0m", result[1]["UserName"], "with", result[1]["NumberOfReviews"], "reviews.",
        "\n - \033[1m3rd\033[0m", result[2]["UserName"], "with", result[2]["NumberOfReviews"], "reviews.")

The three users who wrote the most reviews are: 
 - 1st Sammy with 3756 reviews. 
 - 2nd acurtis with 3403 reviews. 
 - 3rd kylehay2004 with 3368 reviews.


In [80]:
#    3.3. Find all beers that are described with following words: 'fruit', 'complex', 'nutty', 'dark'.
query = """
    MATCH (b:BEERS)
    WHERE toLower(b.notes) CONTAINS 'fruit' OR toLower(b.notes) CONTAINS 'complex' OR toLower(b.notes) CONTAINS 'nutty' OR toLower(b.notes) CONTAINS 'dark'
    RETURN b.name AS BeerName, b.notes AS Description
"""
result = execute_read(driver, query)

print("Beers described with 'fruit', 'complex', 'nutty', and 'dark':")
# for beer in result:
#     print("-\033[1m", beer["BeerName"], "\033[0m:", beer["Description"])
    
print("\n", len(result), "beers found.")

Beers described with 'fruit', 'complex', 'nutty', and 'dark':

 11188 beers found.


<div class="alert alert-block alert-warning">

# **PERGUNTAR PROFª | Não percebi o que é suposto printar**

</div>

In [81]:
#    3.4. Which top three breweries produce the largest variety of beer styles?
query = """
    MATCH (brewery:BREWERIES)-[:BREWED]->(beer:BEERS)-[:HAS_STYLE]->(style:STYLE)
    RETURN brewery.name AS BreweryName, count(DISTINCT style) AS StyleCount
    ORDER BY StyleCount DESC
    LIMIT 3
"""
result = execute_read(driver, query)
print("The top three breweries with the largest variety of beer styles are:",
      "\n - \033[1m1st\033[0m", result[0]["BreweryName"], "with", result[0]["StyleCount"], "styles.",
      "\n - \033[1m2nd\033[0m", result[1]["BreweryName"], "with", result[1]["StyleCount"], "styles.",
      "\n - \033[1m3rd\033[0m", result[2]["BreweryName"], "with", result[2]["StyleCount"], "styles.")

The top three breweries with the largest variety of beer styles are: 
 - 1st Iron Hill Brewery & Restaurant with 94 styles. 
 - 2nd Rock Bottom Restaurant & Brewery with 93 styles. 
 - 3rd Goose Island Beer Co. with 88 styles.


In [82]:
#    3.5. Which country produces the most beer styles?
query = """
    MATCH (c:COUNTRIES)<-[:IN]-(ci:CITIES)<-[:IN]-(br:BREWERIES)-[:BREWED]->(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
    WITH c, count(DISTINCT s) AS NumberOfStyles
    ORDER BY NumberOfStyles DESC
    LIMIT 1
    RETURN c.name AS CountryName, NumberOfStyles
"""
result = execute_read(driver, query)
print("\033[1mThe country producing the most beer styles is:\033[0m", result[0]["CountryName"], "with", result[0]["NumberOfStyles"], "styles.")

The country producing the most beer styles is: US with 113 styles.


---

## **4. Market Analysis department**

In [83]:
# 4. Market Analysis department in your company accesses and updates the trends data on the daily basis. 
#   Given that, consider how you need to optimize the database and its performance so that the following queries are efficient. 
#   Measure performance to communicate your improvements using PROFILE before final query. Answer the following: [4 points]

<div class="alert alert-block alert-warning">

### **Escrever uma nota a dizer que já fizemos certas limpezas que potenciam a otimização das querys seguintes (NAs, Duplicates, Data Types)** (dizer que são exemplos)

</div>

In [14]:
# Function to profile a query and print key performance metrics
# Source: https://neo4j.com/docs/api/python-driver/current/api.html
def profile_query(query, description):
    with driver.session() as session:
        result = session.run(query)
        
        # Consume the summary to get performance metrics
        summary = result.consume()
        
        # Extract key performance metrics
        db_hits = summary.profile["args"]["DbHits"]
        cache_hits = summary.profile["args"]["PageCacheHits"]
        estimated_rows = summary.profile["args"]["EstimatedRows"]
        rows_returned = summary.profile["args"]["Rows"]

        # Print the results for comparison
        print(f"\n🔹 {description} 🔹")
        print(f"Database Hits (DbHits): {db_hits}")
        print(f"Cache Hits (PageCacheHits): {cache_hits}")
        print(f"Estimated Rows: {estimated_rows}")
        print(f"Actual Rows Returned: {rows_returned}\n")

        return {
            "Description": description,
            "DbHits": db_hits,
            "CacheHits": cache_hits,
            "EstimatedRows": estimated_rows,
            "RowsReturned": rows_returned
        }

In [ ]:
# Function to create a INDEX on a property
# https://neo4j.com/docs/cypher-manual/current/indexes/search-performance-indexes/managing-indexes/#list-indexes
def create_index(driver, label, property):
    query = f"CREATE INDEX IF NOT EXISTS FOR (n:{label}) ON (n.{property})"
    return execute_write(driver, query)

In [ ]:
#    4.1. Using ABV score, find 5 strongest beers, display their ABV score and the corresponding brewery? 
#          Keep in mind that the strongest known beer is Snake Venom, and deal with the error entries in the database.

# 4.1. Find the 5 strongest beers by ABV score

# # Before Optimization -                                                                 See in BROWSER
# query = """
#     // PROFILE the query to find the 5 strongest beers by ABV score
#     PROFILE MATCH (br:BREWERIES)-[:BREWED]->(b:BEERS)
#     WHERE b.abv IS NOT NULL
#     RETURN b.name AS beerName, b.abv AS abv, br.name AS breweryName
#     ORDER BY b.abv DESC
#     LIMIT 5
# """
# result = execute_read(driver, query)
# pprint(result)

# # Create an index on the abv property of the BEERS nodes
# query = """
#     CREATE INDEX IF NOT EXISTS FOR (b:BEERS) ON (b.abv)   // Create an index on the abv property of the BEERS nodes
# """                                                       // If we wanna remove it, see the index name in SHOW INDEXES and DROP INDEX index_name
# """
# result = execute_write(driver, query)
# pprint(result)


# # After Optimization
# query = """
#     // PROFILE the query to find the 5 strongest beers by ABV score
#     PROFILE
#     MATCH (br:BREWERIES)-[:BREWED]->(b:BEERS)
#     WHERE b.abv > 0 AND b.abv <= 67.5 AND b.abv IS NOT NULL         // Filter out invalid ABV values (considering the highest valid ABV value)
#     RETURN b.name AS beerName, b.abv AS abv, br.name AS breweryName
#     ORDER BY b.abv DESC
#     LIMIT 5
# """
# result = execute_read(driver, query)
# pprint(result)

In [11]:
# Query BEFORE Optimization
query_before = """
    // PROFILE the query to find the 5 strongest beers by ABV score
    PROFILE MATCH (br:BREWERIES)-[:BREWED]->(b:BEERS)
    WHERE b.abv IS NOT NULL
    RETURN b.name AS beerName, b.abv AS abv, br.name AS breweryName
    ORDER BY b.abv DESC
    LIMIT 5
"""

# Query AFTER Optimization
query_after = """
    PROFILE
    MATCH (br:BREWERIES)-[:BREWED]->(b:BEERS)
    WHERE b.abv > 0 AND b.abv <= 67.5 AND b.abv IS NOT NULL         // Filter out invalid ABV values (considering the highest valid ABV value)
    RETURN b.name AS beerName, b.abv AS abv, br.name AS breweryName
    ORDER BY b.abv DESC
    LIMIT 5
"""

In [15]:
# Run queries and compare performance
before_metrics = profile_query(query_before, "BEFORE Optimization - Find Strongest Beers")
print("\n 1st Result: ")
pprint(execute_read(driver, query_before))
create_index(driver, "BEERS", "abv")
after_metrics = profile_query(query_after, "AFTER Optimization - Find Strongest Beers")
print("\n 2nd Result: ")
pprint(execute_read(driver, query_after))

# Print comparison
print("\n\n🔻 PERFORMANCE COMPARISON 🔻")
for key in before_metrics.keys():
    if key != "Description":
        print(f"{key}: {before_metrics[key]} ➝ {after_metrics[key]}")


🔹 BEFORE Optimization - Find Strongest Beers 🔹
Database Hits (DbHits): 0
Cache Hits (PageCacheHits): 0
Estimated Rows: 5.0
Actual Rows Returned: 5


 1st Result: 
[<Record beerName='Snake Venom' abv=67.5 breweryName='Brewmeister'>,
 <Record beerName='series 3' abv=66.0 breweryName='Redline Brewhouse'>,
 <Record beerName='Armageddon' abv=65.0 breweryName='Brewmeister'>,
 <Record beerName='Start The Future' abv=60.0 breweryName="Brouwerij 't Koelschip">,
 <Record beerName='Schorschbräu Schorschbock 57%' abv=57.5 breweryName='Schorschbräu'>]

🔹 AFTER Optimization - Find Strongest Beers 🔹
Database Hits (DbHits): 0
Cache Hits (PageCacheHits): 0
Estimated Rows: 0.0
Actual Rows Returned: 5


 2nd Result: 
[<Record beerName='Snake Venom' abv=67.5 breweryName='Brewmeister'>,
 <Record beerName='series 3' abv=66.0 breweryName='Redline Brewhouse'>,
 <Record beerName='Armageddon' abv=65.0 breweryName='Brewmeister'>,
 <Record beerName='Start The Future' abv=60.0 breweryName="Brouwerij 't Koelschip"

<center><img src="./img/PROFILEPlan_4.1.png" width="700"></center>

---

In [27]:
#     4.2. Using the answer from question 2 
#               find the top 5 distict beer styles with 
#               the highest average score of smell + feel 
#               that were reviewed by the third most productive user. 
#          Keep in mind that cleaning the database earlier should ensure correct results.

# query = """
#     PROFILE

#     // 1st Match: Find the reviews of the third most productive user (name: 'kylehay2004' based on question 3.2)
#     MATCH (u:USER {name: 'kylehay2004'})<-[:POSTED]-(r:REVIEWS)
#     WHERE r.smell IS NOT NULL AND r.feel IS NOT NULL
    
#     // 2nd Match: Find the beer styles of the reviews
#     MATCH (r)<-[:REVIEWED]-(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
    
#     // 3rd Match: Calculate the average smell + feel score for each style
#     RETURN s.name, AVG(toFloat(r.smell) + toFloat(r.feel)) AS avgSmellFeel       
#     ORDER BY avgSmellFeel DESC
#     LIMIT 5
# """
# result = execute_read(driver, query)
# print("Top 5 styles by smell + feel (unoptimized):")
# pprint(result)

# # Create an index on the name property of the USER nodes & the smell and feel properties of the REVIEWS nodes & the style property of the STYLE nodes
# # Source: https://neo4j.com/docs/cypher-manual/current/indexes/syntax/
# list_of_indexes = [
#     "CREATE INDEX IF NOT EXISTS FOR (u:USER) ON (u.name)",
#     "CREATE INDEX IF NOT EXISTS FOR (r:REVIEWS) ON (r.smell)",
#     "CREATE INDEX IF NOT EXISTS FOR (r:REVIEWS) ON (r.feel)",
#     "CREATE INDEX IF NOT EXISTS FOR (s:STYLE) ON (s.name)"
# ]
# for query in list_of_indexes:
#     result = execute_write(driver, query)
#     pprint(result)

# # After Optimization (Indexes + Query Optimization)
# query = """
#     PROFILE
#     MATCH (u:USER {name: 'kylehay2004'})<-[:POSTED]-(r:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
#     WHERE r.smell IS NOT NULL AND r.feel IS NOT NULL
#     WITH s.name AS styleName, toFloat(r.smell) + toFloat(r.feel) AS smellFeel
#     RETURN styleName, avg(smellFeel) AS avgSmellFeel
#     ORDER BY avgSmellFeel DESC
#     LIMIT 5
# """
# result = execute_read(driver, query)
# print("Top 5 styles by smell + feel (optimized):")
# pprint(result)

In [16]:
# Query BEFORE Optimization
query_before_2 = """
    PROFILE

    // 1st Match: Find the reviews of the third most productive user (name: 'kylehay2004' based on question 3.2)
    MATCH (u:USER {name: 'kylehay2004'})<-[:POSTED]-(r:REVIEWS)
    WHERE r.smell IS NOT NULL AND r.feel IS NOT NULL
    
    // 2nd Match: Find the beer styles of the reviews
    MATCH (r)<-[:REVIEWED]-(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
    
    // 3rd Match: Calculate the average smell + feel score for each style
    RETURN s.name, AVG(toFloat(r.smell) + toFloat(r.feel)) AS avgSmellFeel       
    ORDER BY avgSmellFeel DESC
    LIMIT 5
"""

In [17]:
# Query AFTER Optimization
query_after_2 = """
    PROFILE
    MATCH (u:USER {name: 'kylehay2004'})<-[:POSTED]-(r:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
    WHERE r.smell IS NOT NULL AND r.feel IS NOT NULL
    WITH s.name AS styleName, toFloat(r.smell) + toFloat(r.feel) AS smellFeel
    RETURN styleName, avg(smellFeel) AS avgSmellFeel
    ORDER BY avgSmellFeel DESC
    LIMIT 5
"""

In [18]:
# Run queries and compare performance
before_metrics_2 = profile_query(query_before_2, "BEFORE Optimization - Find Top Beer Styles by Smell + Feel")
print("\n 1st Results:")
pprint(execute_read(driver, query_before_2))
create_index(driver, "USER", "name")
create_index(driver, "REVIEWS", "smell")
create_index(driver, "REVIEWS", "feel")
create_index(driver, "STYLE", "name")
after_metrics_2 = profile_query(query_after_2, "AFTER Optimization - Find Top Beer Styles by Smell + Feel")
print("\n 2nd Results:")
pprint(execute_read(driver, query_after_2))

# Print comparison
print("\n\n🔻 PERFORMANCE COMPARISON 🔻")
for key in before_metrics_2.keys():
    if key != "Description":
        print(f"{key}: {before_metrics_2[key]} ➝ {after_metrics_2[key]}")


🔹 BEFORE Optimization - Find Top Beer Styles by Smell + Feel 🔹
Database Hits (DbHits): 0
Cache Hits (PageCacheHits): 0
Estimated Rows: 5.0
Actual Rows Returned: 5


 1st Results:
[<Record s.name='New England IPA' avgSmellFeel=8.654761904761903>,
 <Record s.name='American Imperial Stout' avgSmellFeel=8.546363636363628>,
 <Record s.name='Bière de Champagne / Bière Brut' avgSmellFeel=8.5>,
 <Record s.name='Belgian Fruit Lambic' avgSmellFeel=8.444444444444445>,
 <Record s.name='Belgian Gueuze' avgSmellFeel=8.42857142857143>]

🔹 AFTER Optimization - Find Top Beer Styles by Smell + Feel 🔹
Database Hits (DbHits): 0
Cache Hits (PageCacheHits): 0
Estimated Rows: 5.0
Actual Rows Returned: 5


 2nd Results:
[<Record styleName='New England IPA' avgSmellFeel=8.654761904761903>,
 <Record styleName='American Imperial Stout' avgSmellFeel=8.546363636363628>,
 <Record styleName='Bière de Champagne / Bière Brut' avgSmellFeel=8.5>,
 <Record styleName='Belgian Fruit Lambic' avgSmellFeel=8.444444444444445>

<center><img src="./img/PROFILEPlan_4.2.png" width="700"></center>

---

## **5. Graph Algorithms**

In [93]:
# 5. Answer **two out of four** of the following questions using Graph Algorithms (gds): 
#        [NB: make sure to clear the graph before using it again] For the quarterly report, Analytics department the follownig information. [6 points]
#     1. Which two countries are most similiar when it comes to their top five most produced Beer styles?
#     2. Which beer is the most popular when considering the number of users who reviewed it? 
#     3. Users are connected together by their reviews of beers, taking into consideration the "smell" score they assign as a weight, 
#        how many communities are formed from these relationships? How many users are in the three largest communities? 
#     4. Which user is the most influential when it comes to reviews of distinct beers by style?

### **5.1. 🌍 Which two countries are most similiar when it comes to their top five most produced Beer styles?**

<div class="alert alert-block alert-danger">

## **CONFIRMAR SE TEMOS DE POR ALGUMA COISA COM `IS NOT NULL`**

- **VERIFICAÇÃO DO JOÃO**

</div>

In [8]:
# Query to search for the top 5 most produced beer styles per country (All countries)
query = """
    MATCH (c:COUNTRIES)<-[:IN]-(city:CITIES)<-[:IN]-(br:BREWERIES)-[:BREWED]->(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
    WITH c, s, COUNT(b) AS beer_count
    ORDER BY c.name, beer_count DESC
    WITH c, COLLECT(s.name)[0..5] AS top_styles  // Get the top 5 styles per country
    RETURN c.name AS country, top_styles
"""

result = execute_read(driver, query)

print("\nTop 5 Beer Styles per Country:")
for rec in result:
    print(f"Country: {rec['country']}, Top Styles: {rec['top_styles']}")


Top 5 Beer Styles per Country:
Country: AD, Top Styles: ['American Blonde Ale', 'German Hefeweizen', 'English India Pale Ale (IPA)', 'Smoke Beer', 'German Doppelbock']
Country: AE, Top Styles: ['Low Alcohol Beer', 'German Helles']
Country: AG, Top Styles: ['American Pale Ale (APA)', 'Fruit and Field Beer', 'American Porter', 'American Light Lager', 'American Adjunct Lager']
Country: AL, Top Styles: ['European Pale Lager', 'Bohemian Pilsener', 'American Amber / Red Lager', 'American Blonde Ale', 'German Maibock']
Country: AM, Top Styles: ['European Pale Lager', 'Smoke Beer', 'Munich Dunkel Lager', 'Bohemian Pilsener', 'American Pale Ale (APA)']
Country: AO, Top Styles: ['European Pale Lager']
Country: AR, Top Styles: ['American IPA', 'American Pale Ale (APA)', 'Belgian Saison', 'American Amber / Red Ale', 'American Wild Ale']
Country: AT, Top Styles: ['German Hefeweizen', 'German Märzen / Oktoberfest', 'European Pale Lager', 'German Pilsner', 'German Kellerbier / Zwickelbier']
Country:

In [9]:
# Existis countries with less than 5 beer styles produced?
query = """
    MATCH (c:COUNTRIES)
    WHERE c.name IS NOT NULL
    WITH COUNT(DISTINCT c) AS total_countries

    MATCH (c:COUNTRIES)<-[:IN]-(city:CITIES)<-[:IN]-(b:BREWERIES)-[:BREWED]->(beer:BEERS)-[:HAS_STYLE]->(s:STYLE)
    WITH c, COUNT(DISTINCT s) AS style_count, total_countries
    WHERE style_count < 5

    WITH COUNT(DISTINCT c) AS low_style_countries, total_countries, 
        COLLECT({country: c.name, style_count: style_count}) AS country_list

    RETURN low_style_countries, total_countries, 
        ROUND((low_style_countries * 100.0) / total_countries, 2) AS percentage, 
        country_list
"""

result = execute_read(driver, query)

# Print results
for rec in result:
    print(f"\nCountries with <5 Beer Styles: {rec['low_style_countries']} out of {rec['total_countries']} ({rec['percentage']}%)\n")
    print("Detailed list:")
    for country in rec['country_list']:
        print(f"{country['country']}: {country['style_count']} beer styles")


Countries with <5 Beer Styles: 54 out of 199 (27.14%)

Detailed list:
IR: 1 beer styles
CG: 4 beer styles
AE: 2 beer styles
TD: 1 beer styles
WS: 3 beer styles
TO: 3 beer styles
ER: 4 beer styles
CK: 3 beer styles
VC: 2 beer styles
SR: 4 beer styles
NE: 1 beer styles
MZ: 4 beer styles
RW: 2 beer styles
NC: 4 beer styles
TM: 2 beer styles
SN: 1 beer styles
AO: 1 beer styles
MA: 4 beer styles
BF: 1 beer styles
GF: 4 beer styles
HT: 2 beer styles
DM: 1 beer styles
VG: 2 beer styles
CI: 3 beer styles
GP: 2 beer styles
ST: 1 beer styles
BW: 1 beer styles
SZ: 3 beer styles
SB: 1 beer styles
CM: 2 beer styles
LS: 1 beer styles
MO: 1 beer styles
SC: 2 beer styles
LY: 1 beer styles
CV: 4 beer styles
TJ: 1 beer styles
PG: 4 beer styles
SS: 1 beer styles
PK: 2 beer styles
YT: 1 beer styles
CF: 1 beer styles
GM: 2 beer styles
IQ: 1 beer styles
GQ: 1 beer styles
PY: 1 beer styles
BJ: 1 beer styles
GW: 1 beer styles
BD: 1 beer styles
GN: 1 beer styles
TG: 3 beer styles
FM: 2 beer styles
GA: 1 beer 

> **Note** It mentions a total of $199$ countries instead of $200$ because we are removing the ghost country without a name

In [10]:
# Distinct beer styles produced by each country 
query = """
    MATCH (c:COUNTRIES)<-[:IN]-(city:CITIES)<-[:IN]-(b:BREWERIES)-[:BREWED]->(beer:BEERS)-[:HAS_STYLE]->(s:STYLE)
    WHERE c.name IS NOT NULL
    WITH c.name AS country, COLLECT(DISTINCT s.name) AS styles
    RETURN country, styles
    ORDER BY size(styles) DESC
"""

result = execute_read(driver, query)

print("\nSample of Countries and Their Beer Styles:")
for rec in result:
    print(f"{rec['country']}: {rec['styles']}")


Sample of Countries and Their Beer Styles:
US: ['German Märzen / Oktoberfest', 'Pumpkin Beer', 'English Sweet / Milk Stout', 'Winter Warmer', 'American Imperial IPA', 'Fruit and Field Beer', 'American Stout', 'American Amber / Red Ale', 'American Lager', 'American Brown Ale', 'American Pale Ale (APA)', 'American Pale Wheat Ale', 'American IPA', 'Bohemian Pilsener', 'Russian Imperial Stout', 'American Wild Ale', 'English Bitter', 'German Altbier', 'English Strong Ale', 'English Pale Ale', 'English India Pale Ale (IPA)', 'English Extra Special / Strong Bitter (ESB)', 'American Cream Ale', 'German Hefeweizen', 'English Oatmeal Stout', 'Scottish Ale', 'Irish Dry Stout', 'German Pilsner', 'Rye Beer', 'American Amber / Red Lager', 'German Dunkelweizen', 'English Porter', 'American Porter', 'American Blonde Ale', 'Belgian Pale Ale', 'Belgian Witbier', 'Scotch Ale / Wee Heavy', 'Belgian Tripel', 'American Barleywine', 'Belgian Dubbel', 'American Black Ale', 'Vienna Lager', 'American Imperial 

<div class="alert alert-block alert-warning">

##### **Applying Node Similarity Algorithm**

Logic followed (Chatgpt)

1. Filters out countries with fewer than 5 beer styles

- Only includes countries with at least 5 distinct beer styles in the graph and have c.name different from NULL
- This ensures that similarity is only computed for countries with a sufficiently diverse beer production.

2. Creates a weighted graph between countries and beer styles

- Each country node (COUNTRIES) is connected to its top 5 most produced beer styles (STYLE).
- The relationship weight is the number of beers in that country belonging to each style.
3. Runs the Node Similarity algorithm using these weighted connections

- Instead of just checking if two countries share beer styles, the algorithm considers how much beer is produced in each style.
- If two countries produce the same styles but in very different proportions, they will have lower similarity.
- If two countries produce the same styles in similar proportions, they will have higher similarity.

> **ALTERAR COMMENT**

</div>

<div class="alert alert-block alert-danger">

# **PERGUNTAR PROFª**

</div>

In [ ]:
# 5.1. Which two countries are most similiar when it comes to their top five most produced Beer styles?
# Step 0 - Clear graph
try:
    query = """CALL gds.graph.drop('beerGraphMostSimilar') YIELD graphName;"""
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

In [97]:
# Step 1 - Create an appropriate graph
try:
    query = """
        CALL gds.graph.project.cypher(
            'beerGraphMostSimilar',

            // Node projection: Include COUNTRIES and STYLE nodes (200 + 113 = 313 nodes)
            "MATCH (n)
             WHERE head(labels(n)) = 'COUNTRIES' OR head(labels(n)) = 'STYLE'
             RETURN id(n) AS id",

            // Relationships: COUNTRIES to top 5 most produced STYLEs
            "MATCH (c:COUNTRIES)<-[:IN]-(:CITIES)<-[:IN]-(:BREWERIES)-[:BREWED]->(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
             WITH c, s, count(b) AS beerCount
             ORDER BY beerCount DESC
             WITH c, collect({styleId: id(s), count: beerCount})[0..5] AS topStyles
             UNWIND topStyles AS styleData
             RETURN id(c) AS source, styleData.styleId AS target"
        )
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    print(f"Error projecting graph: {e}")

[<Record nodeQuery="MATCH (n)\n             WHERE head(labels(n)) = 'COUNTRIES' OR head(labels(n)) = 'STYLE'\n             RETURN id(n) AS id" relationshipQuery='MATCH (c:COUNTRIES)<-[:IN]-(:CITIES)<-[:IN]-(:BREWERIES)-[:BREWED]->(b:BEERS)-[:HAS_STYLE]->(s:STYLE)\n             WITH c, s, count(b) AS beerCount\n             ORDER BY beerCount DESC\n             WITH c, collect({styleId: id(s), count: beerCount})[0..5] AS topStyles\n             UNWIND topStyles AS styleData\n             RETURN id(c) AS source, styleData.styleId AS target' graphName='beerGraphMostSimilar' nodeCount=313 relationshipCount=813 projectMillis=56670>]


In [ ]:
#  graphName='beerGraphMostSimilar' nodeCount=313 relationshipCount=813

In [99]:
# Step 1.2 - Run relationship query
try:
    query = """
        MATCH (c:COUNTRIES)<-[:IN]-(:CITIES)<-[:IN]-(:BREWERIES)-[:BREWED]->(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
        WITH c, s, count(b) AS beerCount
        ORDER BY beerCount DESC
        WITH c, collect({styleId: id(s), count: beerCount})[0..5] AS topStyles
        UNWIND topStyles AS styleData
        RETURN id(c) AS source, styleData.styleId AS target, styleData.count AS count, c.name AS country
        LIMIT 10
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    print(f"Error projecting graph: {e}")

[<Record source=1 target=9494220 count=38566 country='US'>,
 <Record source=1 target=9494228 count=17929 country='US'>,
 <Record source=1 target=9494236 count=16417 country='US'>,
 <Record source=1 target=9494241 count=15182 country='US'>,
 <Record source=1 target=9494237 count=11416 country='US'>,
 <Record source=5 target=9494220 count=4107 country='CA'>,
 <Record source=5 target=9494228 count=2501 country='CA'>,
 <Record source=5 target=9494241 count=2224 country='CA'>,
 <Record source=5 target=9494236 count=1655 country='CA'>,
 <Record source=5 target=9494237 count=1313 country='CA'>]


<div class="alert alert-block alert-warning">

##### **DECIDIR QUAL DOS RESULTADOS POR**

</div>

In [ ]:
# Step 1 - Create an Appropriate Graph
try:
    query = """
    CALL gds.graph.project.cypher(
        'beer_style_graph',

        // Node projection: Include COUNTRIES and STYLE nodes (200 + 113 = 313 nodes)
        "MATCH (n)
        WHERE head(labels(n)) = 'COUNTRIES' OR head(labels(n)) = 'STYLE'
        RETURN id(n) AS id",

        // Relationships: COUNTRIES to top 5 most produced STYLEs (excluding countries with less than 5 styles and without a name)
        "MATCH (c:COUNTRIES)<-[:IN]-(city:CITIES)<-[:IN]-(b:BREWERIES)-[:BREWED]->(beer:BEERS)-[:HAS_STYLE]->(s:STYLE)
        WHERE size([(c)<-[:IN]-(:CITIES)<-[:IN]-(:BREWERIES)-[:BREWED]->(:BEERS)-[:HAS_STYLE]->(:STYLE) | 1]) >= 5 and c.name IS NOT NULL
        WITH c, s, count(beer) AS beerCount
        ORDER BY beerCount DESC
        WITH c, collect({styleId: id(s), count: beerCount})[0..5] AS topStyles
        UNWIND topStyles AS styleData
        RETURN id(c) AS source, styleData.styleId AS target, styleData.count AS weight"
        
    )
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    print(f"Error projecting graph: {e}")

In [ ]:
# Step 2 - Run the Node Similarity Algorithm
try:
    query = """
        CALL gds.nodeSimilarity.stream('beerGraphMostSimilar')
        YIELD node1, node2, similarity
        WITH gds.util.asNode(node1).name AS Country1,
             gds.util.asNode(node2).name AS Country2, similarity
        RETURN Country1, Country2, similarity
        ORDER BY similarity DESC
        LIMIT 2
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    print(f"Error running similarity: {e}")

Most similar countries:
[<Record Country1='US' Country2='PH' similarity=1.0>,
 <Record Country1='US' Country2='JM' similarity=1.0>]


In [ ]:
print("\033[1mMost Similar Countries Based on Beer Styles:\033[1m")
for rec in result:
    print(f"{rec['Country1']} and {rec['Country2']} - Similarity: {rec['similarity']:.2f}")

#### **Comments**

- The similarity between countries based on their top five most produced beer styles can be calculated using the Jaccard Similarity Coefficient. This coefficient measures the similarity between two sets by dividing the size of the intersection by the size of the union of the two sets.

**Results**



---

### **5.2. 🍺 Which beer is the most popular when considering the number of users who reviewed it?**

In [37]:
# 5.2. Which beer is the most popular when considering the number of users who reviewed it?

# Step 0 - Clear graph
try:
    query = """CALL gds.graph.drop('beerGraphMostPopular') YIELD graphName;"""
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record graphName='beerGraphMostPopular'>]


In [38]:
# Step 1 - Create an appropriate graph
try:
    query = """
        CALL gds.graph.project.cypher(
            'beerGraphMostPopular',

            // Node projection: Include BEERS and USER nodes
            "MATCH (n)
             WHERE head(labels(n)) = 'BEERS' OR head(labels(n)) = 'USER'
             RETURN id(n) AS id",

            // Relationships: BEERS to USERs who reviewed them
            "MATCH (b:BEERS)-[:REVIEWED]->(r:REVIEWS)-[:POSTED]->(u:USER)
             RETURN id(b) AS source, id(u) AS target"
        )
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    print(f"Error creating graph: {e}")

[<Record nodeQuery="MATCH (n)\n             WHERE head(labels(n)) = 'BEERS' OR head(labels(n)) = 'USER'\n             RETURN id(n) AS id" relationshipQuery='MATCH (b:BEERS)-[:REVIEWED]->(r:REVIEWS)-[:POSTED]->(u:USER)\n             RETURN id(b) AS source, id(u) AS target' graphName='beerGraphMostPopular' nodeCount=482804 relationshipCount=2537991 projectMillis=554328>]


In [ ]:
# graphName='beerGraphMostPopular' nodeCount=482804 relationshipCount=2537991

In [47]:
# Step 1.1 - Verify the graph has only 1 relationship between BEERS and USER
try:
    query = """
        MATCH (b:BEERS)-[:REVIEWED]->(r:REVIEWS)-[:POSTED]->(u:USER)
        WITH b, u, count(r) AS reviewCount
        WHERE reviewCount > 1
        RETURN b.id AS beerId, u.name AS userName, reviewCount AS relationshipCount
        LIMIT 5
    """
    result = execute_read(driver, query)
    if not result:
        print("Verification: Only 1 relationship exists between each BEERS and USER pair in the original data.")
    else:
        print("Verification: Multiple reviews found in original data (expected, but graph should deduplicate):")
        pprint(result)
except Exception as e:
    print(f"Error verifying graph: {e}")

Verification: Multiple reviews found in original data (expected, but graph should deduplicate):
[<Record beerId=41626 userName=None relationshipCount=2>,
 <Record beerId=64228 userName=None relationshipCount=3>,
 <Record beerId=55081 userName=None relationshipCount=4>,
 <Record beerId=35754 userName=None relationshipCount=2>,
 <Record beerId=33832 userName=None relationshipCount=2>]


In [51]:
# Step 1.1 - Verify the graph has only 1 relationship between BEERS and USER
try:
    query = """
        MATCH (b:BEERS)-[:REVIEWED]->(r:REVIEWS)-[:POSTED]->(u:USER)
        WITH b, u, count(r) AS reviewCount
        WHERE reviewCount > 1 AND b.id IS NOT NULL AND u.name IS NOT NULL
        RETURN b.id AS beerId, u.name AS userName, reviewCount AS relationshipCount
        LIMIT 5
    """
    result = execute_read(driver, query)
    if not result:
        print("Verification: Only 1 relationship exists between each BEERS and USER pair in the original data.")
    else:
        print("Verification: Multiple reviews found in original data (expected, but graph should deduplicate):")
        pprint(result)
except Exception as e:
    print(f"Error verifying graph: {e}")

Verification: Only 1 relationship exists between each BEERS and USER pair in the original data.


In [ ]:
# Step 1.2. Not consider more than one review per user for the same beer
query = """
    MATCH (b:BEERS)-[:REVIEWED]->(r:REVIEWS)-[:POSTED]->(u:USER)
    WITH DISTINCT b, u, count(r) AS reviewCount
    WHERE reviewCount > 1 
    RETURN b.id AS beerId, u.name AS userName, reviewCount AS relationshipCount
    LIMIT 5
"""
result = execute_read(driver, query)
pprint(result)

[<Record beerId=41626 userName=None relationshipCount=2>,
 <Record beerId=64228 userName=None relationshipCount=3>,
 <Record beerId=55081 userName=None relationshipCount=4>,
 <Record beerId=35754 userName=None relationshipCount=2>,
 <Record beerId=33832 userName=None relationshipCount=2>]


In [ ]:
# Step 2 - Run a centrality algorithm to find the most popular beer
# Source: https://neo4j.com/docs/graph-data-science/current/algorithms/degree-centrality/
try:
    query = """
        CALL gds.degree.stream('beerGraphMostPopular')
        YIELD nodeId, score
        WITH gds.util.asNode(nodeId).name AS BeerName, score AS NumberOfReviews
        RETURN BeerName, NumberOfReviews
        ORDER BY NumberOfReviews DESC
        LIMIT 10
    """
    result = execute_read(driver, query)
    print("Most popular beer by number of users who reviewed it:")
    pprint(result)
except Exception as e:
    print(f"Error running centrality: {e}")

Most popular beer by number of users who reviewed it:
[<Record BeerName='Breakfast Stout' NumberOfReviews=4833.0>,
 <Record BeerName='90 Minute IPA' NumberOfReviews=4394.0>,
 <Record BeerName='Pliny The Elder' NumberOfReviews=4218.0>,
 <Record BeerName='Two Hearted Ale' NumberOfReviews=4174.0>,
 <Record BeerName='Heady Topper' NumberOfReviews=3974.0>,
 <Record BeerName='Sculpin IPA' NumberOfReviews=3962.0>,
 <Record BeerName='Hopslam Ale' NumberOfReviews=3844.0>,
 <Record BeerName='KBS (Kentucky Breakfast Stout)' NumberOfReviews=3711.0>,
 <Record BeerName='Bourbon County Brand Stout' NumberOfReviews=3615.0>,
 <Record BeerName='Old Rasputin' NumberOfReviews=3531.0>]


In [43]:
# Query to find the most popular beer by the number of users who reviewed it
query = """
    MATCH (b:BEERS)-[:REVIEWED]->(r:REVIEWS)-[:POSTED]->(u:USER)
    RETURN b.name AS BeerName, count(DISTINCT u) AS NumberOfUsers
    ORDER BY NumberOfUsers DESC
    LIMIT 5
"""
result = execute_read(driver, query)
print("Most popular beer by number of users who reviewed it:")
pprint(result)

Most popular beer by number of users who reviewed it:
[<Record BeerName='IPA' NumberOfUsers=6884>,
 <Record BeerName='Breakfast Stout' NumberOfUsers=4930>,
 <Record BeerName='Porter' NumberOfUsers=4737>,
 <Record BeerName='90 Minute IPA' NumberOfUsers=4394>,
 <Record BeerName='Imperial Stout' NumberOfUsers=4263>]


In [ ]:
# Query to find the most popular beer by the number of users who reviewed it
query = """
    MATCH (b:BEERS)-[:REVIEWED]->(r:REVIEWS)-[:POSTED]->(u:USER)
    WITH DISTINCT b, u
    WHERE b.id IS NOT NULL AND u.name IS NOT NULL
    RETURN b.id AS beerId, u.name AS userName, count(u) AS reviewCount
    ORDER BY reviewCount DESC
    LIMIT 5
"""
result = execute_read(driver, query)
print("Most popular beer by number of users who reviewed it:")
pprint(result)

Most popular beer by number of users who reviewed it:
[<Record beerId=52211 userName='bluejacket74' reviewCount=1>,
 <Record beerId=86149 userName='bluejacket74' reviewCount=1>,
 <Record beerId=104855 userName='bluejacket74' reviewCount=1>,
 <Record beerId=143877 userName='bluejacket74' reviewCount=1>,
 <Record beerId=61128 userName='bluejacket74' reviewCount=1>]


<div class="alert alert-block alert-danger">

## **🚨NÃO DEU IGUAL À QUERY SIMPLES ACIMA! 🚨**

---

### **5.4. 📝 Which user is the most influential when it comes to reviews of distinct beers by style?**

In [103]:
# Step 0 - Clear graph
try:
    query = """CALL gds.graph.drop('beerReviewGraphMostInfluential') YIELD graphName;"""
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

ClientError('Failed to invoke procedure `gds.graph.drop`: Caused by: java.util.NoSuchElementException: Graph with name `beerReviewGraphMostInfluential` does not exist on database `neo4j`. It might exist on another database.')


In [104]:
# Number of distinct beers reviewed by users
query = """
    MATCH (u:USER)<-[:POSTED]-(:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
    WITH u, s, count(DISTINCT b) AS distinctBeers
    RETURN count(DISTINCT s) AS NumberOfStyles, count(DISTINCT u) AS NumberOfUsers, sum(distinctBeers) AS TotalDistinctBeers
"""
result = execute_read(driver, query)
pprint(result)

[<Record NumberOfStyles=113 NumberOfUsers=106645 TotalDistinctBeers=2537753>]


In [105]:
# Step 1 - Create the Graph
try:
    query = """
        CALL gds.graph.project.cypher(
            'beerReviewGraphMostInfluential',
            
            // Nodes: USER and STYLE
            'MATCH (n) WHERE n:USER OR n:STYLE RETURN id(n) AS id',
            
             // Relationships: USER to STYLE via distinct beers reviewed
            'MATCH (u:USER)<-[:POSTED]-(:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
             WITH u, s, count(DISTINCT b) AS distinctBeers
             RETURN id(u) AS source, id(s) AS target, distinctBeers AS weight'
        )
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record nodeQuery='MATCH (n) WHERE n:USER OR n:STYLE RETURN id(n) AS id' relationshipQuery='MATCH (u:USER)<-[:POSTED]-(:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[:HAS_STYLE]->(s:STYLE)\n             WITH u, s, count(DISTINCT b) AS distinctBeers\n             RETURN id(u) AS source, id(s) AS target, distinctBeers AS weight' graphName='beerReviewGraphMostInfluential' nodeCount=124048 relationshipCount=893394 projectMillis=112683>]


In [ ]:
# graphName='beerReviewGraphMostInfluential' nodeCount=124_048 relationshipCount=893_394

In [41]:
# Query to calculate user based on the number of distinct beers reviewed by style
query = """
    MATCH (u:USER)<-[:POSTED]-(:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
    WITH u, s, count(DISTINCT b) AS distinctBeers
    RETURN id(u) AS source, id(s) AS target, distinctBeers AS weight
    ORDER BY weight DESC
    LIMIT 10
"""
result = execute_read(driver, query)
pprint(result)

[<Record source=9494459 target=9494220 weight=387>,
 <Record source=9494470 target=9494220 weight=382>,
 <Record source=9495389 target=9494220 weight=379>,
 <Record source=9495292 target=9494220 weight=367>,
 <Record source=9494633 target=9494220 weight=360>,
 <Record source=9494459 target=9494241 weight=352>,
 <Record source=9495151 target=9494220 weight=342>,
 <Record source=9494459 target=9494237 weight=337>,
 <Record source=9494682 target=9494220 weight=326>,
 <Record source=9494767 target=9494220 weight=322>]


In [107]:
# Step 2: Run the PageRank Algorithm
try:
    query = """
        CALL gds.pageRank.stream('beerReviewGraphMostInfluential', {relationshipWeightProperty: 'weight'})
        YIELD nodeId, score
        WITH gds.util.asNode(nodeId).name AS username, score
        RETURN username, score
        ORDER BY score DESC
        LIMIT 10
    """
    result = execute_read(driver, query)
    print("Most influential users by beer reviews across styles:")
    pprint(result)
except Exception as e:
    pprint(e)

Most influential users by beer reviews across styles:
[<Record username='American IPA' score=1837.8293276018699>,
 <Record username='American Imperial IPA' score=1337.6323127285552>,
 <Record username='American Imperial Stout' score=750.5777123781654>,
 <Record username='American Pale Ale (APA)' score=631.3762677718211>,
 <Record username='Russian Imperial Stout' score=356.3576606152439>,
 <Record username='American Porter' score=302.40946886296024>,
 <Record username='Fruit and Field Beer' score=300.3043547353978>,
 <Record username='Belgian Saison' score=296.9869531999581>,
 <Record username='New England IPA' score=284.96372700942555>,
 <Record username='American Amber / Red Ale' score=268.0152379580384>]


<div class="alert alert-block alert-danger">

# **PERGUNTAR PROFª**

</div>

#### **Comments**

---

## **🔗 Bibliography/References**

**[[1]](https://www.hopsandhopes.nl/en/beers/snake-venom)** Hops & Hopes. (2025). Snake Venom. Nederlands; Hops&Hopes. https://www.hopsandhopes.nl/en/beers/snake-venom